# <font color = 'color'>[R텍마08] 2부4장_텍스트 데이터 전처리: tm패키지 이용</font>

##### 필요 패키지 인스톨 및 라이브러리 로딩

In [1]:
# 필요한 패키지 목록 생성
pkg <- c('stringr', 'magrittr', 'purrr', 'dplyr', 'tm')

# 필요패키지 설지여부를 체크해 미설치패키지 목록을 저장
new_pkg <- pkg[!(pkg %in% rownames(installed.packages()))]

# 미설치 패키지 목록이 1개라도 있으면, 일괄 인스톨 실시
if (length(new_pkg)) install.packages(new_pkg, dependencies = TRUE, repos="http://R-Forge.R-project.org")

# 필요패키지를 일괄 로딩실시
suppressMessages(sapply(pkg, require, character.only = TRUE))  

Warning message:
"dependencies 'NLP', 'slam', 'antiword', 'filehash', 'pdftools', 'Rcampdf', 'Rgraphviz', 'Rpoppler', 'SnowballC', 'tm.lexicon.GeneralInquirer' are not available"

package 'tm' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\student\AppData\Local\Temp\RtmpSUur28\downloaded_packages


Warning message:
"package 'tm' was built under R version 3.5.2"

stringr magrittr    purrr    dplyr       tm 
    TRUE     TRUE     TRUE     TRUE    FALSE

# <font color = 'blue'>텍스트셋 준비 (p.113)</font>
* 영어로 작성된 여러 개의 논문초록(abstract) 파일들을 하나의 말뭉치(Corpus)로 만들어 분석수행
* 이를 위해 작업경로내 관련 폴더와 파일 정보를 확인함

## 작업경로내 텍스트셋 정보 확인

### 폴더 정보 확인

In [2]:
# 현재 작업경로 확인
getwd() %>% print

[1] "C:/my/OneDrive/mcpy/tm"


In [3]:
# 작업경로내 폴더목록과 파일목록 조회
dir() %>% print
# - 서브폴더와 서브폴더내 파일목록은 보이지 않음

 [1] "[R텍마01] 2부1장_텍스트분석을 위한 R기초.ipynb"                 
 [2] "[R텍마02] 2부1장_텍스트분석을 위한 데이터순환 함수.ipynb"       
 [3] "[R텍마03] 2부2장_텍스트분석을 위한 R 기본함수.ipynb"            
 [4] "[R텍마04] 2부2장_텍스트분석을 위한 정규표현식.ipynb"            
 [5] "[R텍마05] 2부2장3장_텍스트분석을 위한 정규표현식2.ipynb"        
 [6] "[R텍마06] 2부2장3장_텍스트분석을 위한 정규표현식3.ipynb"        
 [7] "[R텍마07] 2부2장3장_텍스트분석을 위한 정규표현식4.ipynb"        
 [8] "[R텍마08] 2부4장_텍스트 데이터 전처리.ipynb"                    
 [9] "[R텍마09] 2부4장_텍스트 데이터 전처리_tm패키지 이용-Copy1.ipynb"
[10] "[R텍마09] 2부4장_텍스트 데이터 전처리_tm패키지 이용.ipynb"      
[11] "datatm"                                                         
[12] "images"                                                         


In [4]:
# 작업경로내 폴더목록만 조회
list.dirs(recursive = F) %>% print

[1] "./.ipynb_checkpoints" "./datatm"             "./images"            


In [5]:
# 작업경로내 폴더목록과 각 폴더별 하위 폴더목록까지 조회
list.dirs(recursive = T) %>% print

[1] "."                         "./.ipynb_checkpoints"     
[3] "./datatm"                  "./datatm/ymbaek_papers_en"
[5] "./images"                 


In [6]:
# 특정 패턴을 갖는 폴더&파일 목록 존재여부 체크
file.exists("data") %>% print

# 특정한 패턴을 갖는 폴더&파일 목록 조회
dir(pattern = 'data') %>% print

[1] FALSE
[1] "datatm"


In [7]:
# 특정 폴더내 서브폴더 목록 조회
list.dirs(path = './datatm') %>% print

[1] "./datatm"                  "./datatm/ymbaek_papers_en"


### 파일 정보 확인

In [8]:
# 작업경로내 파일목록과 폴더목록 조회
list.files(recursive = F) %>% print

 [1] "[R텍마01] 2부1장_텍스트분석을 위한 R기초.ipynb"                 
 [2] "[R텍마02] 2부1장_텍스트분석을 위한 데이터순환 함수.ipynb"       
 [3] "[R텍마03] 2부2장_텍스트분석을 위한 R 기본함수.ipynb"            
 [4] "[R텍마04] 2부2장_텍스트분석을 위한 정규표현식.ipynb"            
 [5] "[R텍마05] 2부2장3장_텍스트분석을 위한 정규표현식2.ipynb"        
 [6] "[R텍마06] 2부2장3장_텍스트분석을 위한 정규표현식3.ipynb"        
 [7] "[R텍마07] 2부2장3장_텍스트분석을 위한 정규표현식4.ipynb"        
 [8] "[R텍마08] 2부4장_텍스트 데이터 전처리.ipynb"                    
 [9] "[R텍마09] 2부4장_텍스트 데이터 전처리_tm패키지 이용-Copy1.ipynb"
[10] "[R텍마09] 2부4장_텍스트 데이터 전처리_tm패키지 이용.ipynb"      
[11] "datatm"                                                         
[12] "images"                                                         


In [9]:
# 작업경로내 파일목록과 각 폴더별 파일목록을 일괄 조회
list.files(recursive = T) %>% print

 [1] "[R텍마01] 2부1장_텍스트분석을 위한 R기초.ipynb"                 
 [2] "[R텍마02] 2부1장_텍스트분석을 위한 데이터순환 함수.ipynb"       
 [3] "[R텍마03] 2부2장_텍스트분석을 위한 R 기본함수.ipynb"            
 [4] "[R텍마04] 2부2장_텍스트분석을 위한 정규표현식.ipynb"            
 [5] "[R텍마05] 2부2장3장_텍스트분석을 위한 정규표현식2.ipynb"        
 [6] "[R텍마06] 2부2장3장_텍스트분석을 위한 정규표현식3.ipynb"        
 [7] "[R텍마07] 2부2장3장_텍스트분석을 위한 정규표현식4.ipynb"        
 [8] "[R텍마08] 2부4장_텍스트 데이터 전처리.ipynb"                    
 [9] "[R텍마09] 2부4장_텍스트 데이터 전처리_tm패키지 이용-Copy1.ipynb"
[10] "[R텍마09] 2부4장_텍스트 데이터 전처리_tm패키지 이용.ipynb"      
[11] "datatm/ymbaek_papers_en/p2009a.txt"                             
[12] "datatm/ymbaek_papers_en/p2009b.txt"                             
[13] "datatm/ymbaek_papers_en/p2010a.txt"                             
[14] "datatm/ymbaek_papers_en/p2010b.txt"                             
[15] "datatm/ymbaek_papers_en/p2010c.txt"                             
[16] "datatm/ymbaek_papers_en/p2011a.txt"                             
[17] "datatm/ymbaek_papers_en/p20

In [10]:
# 특정 폴더내 서브폴더 목록 조회
list.dirs(path = './datatm') %>% print
cat('\n')
# 특정 폴더내 파일 목록 조회
list.files (path = "./datatm/ymbaek_papers_en") %>% print

[1] "./datatm"                  "./datatm/ymbaek_papers_en"

 [1] "p2009a.txt" "p2009b.txt" "p2010a.txt" "p2010b.txt" "p2010c.txt"
 [6] "p2011a.txt" "p2011b.txt" "p2012a.txt" "p2012b.txt" "p2013a.txt"
[11] "p2014a.txt" "p2014b.txt" "p2014c.txt" "p2014d.txt" "p2014e.txt"
[16] "p2014f.txt" "p2014g.txt" "p2014h.txt" "p2014i.txt" "p2014k.txt"
[21] "p2015a.txt" "p2015b.txt" "p2015c.txt" "p2015d.txt"


# <font color = 'blue'>말뭉치(Corpus) 생성 (p.115)</font>
* 특정한 폴더에 저장되어 있는 텍스트셋을 로딩하여, 말뭉치 생성작업을 실시함
* 준비된 텍스트 파일이 1개면 1개 문서로 말뭉치를 구성함
* 여러 개 텍스트 파일인 경우 각 문서의 독립성을 유지하면서 하나의 말뭉치 형태로 접근할 수 있도록 리스트 구조를 가지게됨 

## 텍스트셋 파일로부터 말뭉치 생성

In [11]:
# 말뭉치 생성대상 텍스트데이터 위치경로 설정
loc_papers_en <- './datatm/ymbaek_papers_en'

# tm패키지 Vcorpus() 함수를 이용해 텍스트셋 파일들을 통합해 말뭉치로 생성함
my <-VCorpus(DirSource(loc_papers_en))

## 말뭉치 기본 특성파악

In [12]:
# 말뭉치 간단출력
my %>% print
# - Metadata: 말뭉치를 구성하는 각 파일에 대한 별도의 설명은 없는 상태임
# - Content: 24개 문서(documents)로 구성되어 있음을 알 수 있음

sprintf('말뭉치 객체유형:')
class(my) %>% print
# - Corpus객체도 리스트 객체구조를 가지고 있는데, 
#   결합되는 여러개 텍스트셋 파일별 내용과 메타정보를 독립적으로 각 리스트항목으로 유지하는 구조임
# - VCorpus: Volatile Corpus 임시생성(휘발성있는) 코퍼스 파일

sprintf('말뭉치 구성요소 길이: %d', length(my))  %>% print

<<VCorpus>>
Metadata:  corpus specific: 0, document level (indexed): 0
Content:  documents: 24


[1] "말뭉치 객체유형:"

[1] "VCorpus" "Corpus" 
[1] "말뭉치 구성요소 길이: 24"


In [13]:
# 말뭉치 객체의 리스트항목이름 조회
names(my) %>% print
# - 여러개의 텍스트파일이 하나의 말뭉치로 결합되면서 
#   각 파일명이 리스트항목명으로 반영되었음을 알 수 있음 

 [1] "p2009a.txt" "p2009b.txt" "p2010a.txt" "p2010b.txt" "p2010c.txt"
 [6] "p2011a.txt" "p2011b.txt" "p2012a.txt" "p2012b.txt" "p2013a.txt"
[11] "p2014a.txt" "p2014b.txt" "p2014c.txt" "p2014d.txt" "p2014e.txt"
[16] "p2014f.txt" "p2014g.txt" "p2014h.txt" "p2014i.txt" "p2014k.txt"
[21] "p2015a.txt" "p2015b.txt" "p2015c.txt" "p2015d.txt"


In [14]:
# 말뭉치 객체의 간략요약정보 확인
summary(my)
# - 각 문서들은 PlainTextDocument 포맷양식이며, 리스트 구조임 
# - 각 문서별로 리스트객체의 길이(Length)가 2개인데, 
#   1개는 문서내용($content)이며, 또 다른 1개는 해당 문서에 대한 메타($meta) 정보임

           Length Class             Mode
p2009a.txt 2      PlainTextDocument list
p2009b.txt 2      PlainTextDocument list
p2010a.txt 2      PlainTextDocument list
p2010b.txt 2      PlainTextDocument list
p2010c.txt 2      PlainTextDocument list
p2011a.txt 2      PlainTextDocument list
p2011b.txt 2      PlainTextDocument list
p2012a.txt 2      PlainTextDocument list
p2012b.txt 2      PlainTextDocument list
p2013a.txt 2      PlainTextDocument list
p2014a.txt 2      PlainTextDocument list
p2014b.txt 2      PlainTextDocument list
p2014c.txt 2      PlainTextDocument list
p2014d.txt 2      PlainTextDocument list
p2014e.txt 2      PlainTextDocument list
p2014f.txt 2      PlainTextDocument list
p2014g.txt 2      PlainTextDocument list
p2014h.txt 2      PlainTextDocument list
p2014i.txt 2      PlainTextDocument list
p2014k.txt 2      PlainTextDocument list
p2015a.txt 2      PlainTextDocument list
p2015b.txt 2      PlainTextDocument list
p2015c.txt 2      PlainTextDocument list
p2015d.txt 2    

In [15]:
# 말뭉치 객체의 내부구조정보 확인
str(my)
# - 말뭉치 자체가 24개 텍스트문서로 구성된 리스트객체임
# - 말뭉치에 결합된 각 문서들은 PlainTextDocument 포맷양식이며, 또한 리스트 구조임 
# - 즉 말뭉치를 구성하는 각 리스트항목(문서) 별로 세부 리스트객체의 길이(Length)가 2개인데, 
#   1개는 문서내용($content)이며, 또 다른 1개는 해당 문서에 대한 메타($meta) 정보를 가지고 있는 구조임
# - 메타항목을 보면 각 파일별 id(파일명)와 생성시점(datetimestamp), 사용언어(language)가 확인되며, 
#   다른 메타항목은 별도 정보가 없는 상태임

# 주의
# - 각 리스트항목(문서)별 세부 리스트항목 2개 요소 중에서 $content를 보면 
#   24개 파일중 23개 파일에서는 content의 구성내용이 1개 문자열임을 알 수 있음
# - 그런데, 13번째 파일인 p2014c.txt에 대한 리스트항목의 $content를 보면,
#   $ content: chr [1:8]로 나타나 있어, 세부적으로 8개 문자열 요소로 구성되어 있음을 알 수 있음
# ==> 말뭉치에 대한 전처리 작업시 각 리스트항목별로 구성요소갯수가 달라서 오류가 발생하므로
#     모든 리스트항목(문서) 별로 동일한 조건인 1개 문자열 상태로 만들어 주어야 함

List of 24
 $ p2009a.txt:List of 2
  ..$ content: chr "Does deliberative setting, online versus face-to-face, influence citizens' experiences? Are certain factors dif"| __truncated__
  ..$ meta   :List of 7
  .. ..$ author       : chr(0) 
  .. ..$ datetimestamp: POSIXlt[1:1], format: "2019-01-18 00:08:01"
  .. ..$ description  : chr(0) 
  .. ..$ heading      : chr(0) 
  .. ..$ id           : chr "p2009a.txt"
  .. ..$ language     : chr "en"
  .. ..$ origin       : chr(0) 
  .. ..- attr(*, "class")= chr "TextDocumentMeta"
  ..- attr(*, "class")= chr [1:2] "PlainTextDocument" "TextDocument"
 $ p2009b.txt:List of 2
  ..$ content: chr "The debate on late night comedy has been inconclusive, with some scholars arguing that this genre increases pol"| __truncated__
  ..$ meta   :List of 7
  .. ..$ author       : chr(0) 
  .. ..$ datetimestamp: POSIXlt[1:1], format: "2019-01-18 00:08:01"
  .. ..$ description  : chr(0) 
  .. ..$ heading      : chr(0) 
  .. ..$ id           : chr "p2009b.txt"
  .

In [16]:
# 말뭉치 객체의 각 문서별 content 항목의 세부요소갯수 파악 
sapply(my, lengths) %>% print %>% class
# - 말뭉치를 구성하고 있는 각 리스트항목(문서)에 데이터순환방식으로 접근해 
#   content 항목의 길이를 카운팅하고, 결과를 행렬구조로 보여줌
# - 대부분 문서파일의 content항목의 길이는 1로 나와서 1개의 문자열로 구성되어 있지만,
#   p2014c.txt 파일에 들어 있는 content항목의 길이는 8개로 파악되고 있어
#   이를 unlist를 적용해 1개 문자열로 만들어야 함

        p2009a.txt p2009b.txt p2010a.txt p2010b.txt p2010c.txt p2011a.txt
content          1          1          1          1          1          1
meta             7          7          7          7          7          7
        p2011b.txt p2012a.txt p2012b.txt p2013a.txt p2014a.txt p2014b.txt
content          1          1          1          1          1          1
meta             7          7          7          7          7          7
        p2014c.txt p2014d.txt p2014e.txt p2014f.txt p2014g.txt p2014h.txt
content          8          1          1          1          1          1
meta             7          7          7          7          7          7
        p2014i.txt p2014k.txt p2015a.txt p2015b.txt p2015c.txt p2015d.txt
content          1          1          1          1          1          1
meta             7          7          7          7          7          7


[1] "matrix"

In [17]:
# 말뭉치 객체의 각 문서별 content 항목의 세부요소갯수 파악
lapply(my, lengths) %>% print %>% class
# - 말뭉치를 구성하고 있는 각 리스트항목(문서)에 데이터순환방식으로 접근해 
#   content 항목의 길이를 카운팅하고, 결과를 리스트구조로 보여줌
# - 대부분 문서파일의 content항목의 길이는 1로 나와서 1개의 문자열로 구성되어 있지만,
#   p2014c.txt 파일에 들어 있는 content항목의 길이는 8개로 파악되고 있어
#   이를 unlist를 적용해 1개 문자열로 만들어야 함

$p2009a.txt
content    meta 
      1       7 

$p2009b.txt
content    meta 
      1       7 

$p2010a.txt
content    meta 
      1       7 

$p2010b.txt
content    meta 
      1       7 

$p2010c.txt
content    meta 
      1       7 

$p2011a.txt
content    meta 
      1       7 

$p2011b.txt
content    meta 
      1       7 

$p2012a.txt
content    meta 
      1       7 

$p2012b.txt
content    meta 
      1       7 

$p2013a.txt
content    meta 
      1       7 

$p2014a.txt
content    meta 
      1       7 

$p2014b.txt
content    meta 
      1       7 

$p2014c.txt
content    meta 
      8       7 

$p2014d.txt
content    meta 
      1       7 

$p2014e.txt
content    meta 
      1       7 

$p2014f.txt
content    meta 
      1       7 

$p2014g.txt
content    meta 
      1       7 

$p2014h.txt
content    meta 
      1       7 

$p2014i.txt
content    meta 
      1       7 

$p2014k.txt
content    meta 
      1       7 

$p2015a.txt
content    meta 
      1       7 

$p2015b.txt
c

[1] "list"

## 말뭉치 세부 내용 파악

### 말뭉치 문서별 특성파악 

In [18]:
# 말뭉치 중 특정 문서에 대한 기본특성조회
# - 2가지 방식으로 접근가능함
#   (1) 결합된 순서대로 부여받는 숫자인덱스를 이용한 방식
#   (2) 문서결합시 리스트항목병으로 자동설정된 문서명으로 인덱싱 

my[[1]]
my[['p2009a.txt']] 
# - 기본 특성내용 파악: 메타데이터 항목이 7개, 글자수는 990글자

my[[13]]
my[['p2014c.txt']]
# - 기본 특성내용 파악: 메타데이터 항목이 7개, 글자수는 990글자

<<PlainTextDocument>>
Metadata:  7
Content:  chars: 993

<<PlainTextDocument>>
Metadata:  7
Content:  chars: 993

<<PlainTextDocument>>
Metadata:  7
Content:  chars: 1386

<<PlainTextDocument>>
Metadata:  7
Content:  chars: 1386

In [19]:
# 말뭉치를 특정문서 세부항목 확인
# - 말뭉치를 구성하는 리스트항목(특정문서)별로 
#   content와 meta라는 세부항목을 보유하고 있음을 알 수 있음 

names(my[[1]]) %>% print 
names(my[['p2009a.txt']] ) %>% print

names(my[[13]]) %>% print
names(my[['p2014c.txt']]) %>% print

[1] "content" "meta"   
[1] "content" "meta"   
[1] "content" "meta"   
[1] "content" "meta"   


In [20]:
# 말뭉치 중 특정문서의 내용(content)확인
my[[1]]$content %>% print
# - 작은따옴표로 열고/닫은 형태의 단일한 1개의 문자열 데이터임을 알 수 있음

my[[1]]$content %>% length
# - 작은따옴표로 열고/닫은 형태의 단일한 1개의 문자열이므로 길이도 1로 나옴

# 말뭉치 중 특정문서의 글자수(nuber of character)확인
my[[1]]$content %>% str_length %>% print
# - 1개 문자열이 가지고 있는 글자수

[1] "Does deliberative setting, online versus face-to-face, influence citizens' experiences? Are certain factors differently influential in one setting than in the other? We draw on a nationally representative survey and identify  citizens who participated in both online and face-to-face settings (n  = 82). We use structural equation modeling to first assess the effects that deliberation format has on the interrelationship between such crucial factors as motivations to deliberate, perceived diversity, elicited emotions, enhanced understanding, and goal evaluation. We later employ network analysis to ask which factor or which cluster of factors is more central to an overall experience in which format. Relying on citizens who participated in both settings and using within-subject analyses, we  assure that the previously unnoted findings are attributable to the format per se rather than to individual characteristics. We discuss the theoretical, practical and methodological implications."


[1] 1

[1] 993


In [21]:
# 말뭉치 중 특정문서의 내용(content)확인
my[[13]]$content %>% print
# - 작음따옴표로 열고/닫은 형태가 여러개로 관찰되는 데이터임을 알 수 있음

my[[13]]$content %>% length
# - 작은따옴표로 열고/닫은 형태가 여러 개로 관찰되므로 길이도 8로 나옴

# 말뭉치 중 특정문서의 글자수(nuber of character)확인
my[[13]]$content %>% str_length %>% print
# - 각 문자열이 가지고 있는 각각의 글자수

[1] "BACKGROUND AND AIMS:"                                                                                                                                                                                                                                                                                                                                                                                                 
[2] "Little is known about the extent depression adds to the costs of treatment for physical health conditions. This study examined the paths and the extent to which depression in conjunction with a physical health problem is associated with an increase in healthcare expenditures and how that is different between younger and older adults."                                                                      
[3] "METHODS:"                                                                                                                                                                  

[1] 8

[1]  20 335   8 270   8 405  12 328


In [22]:
# 말뭉치 중 특정문서의 메타정보(meta)확인
my[[1]]$meta

  author       : character(0)
  datetimestamp: 2019-01-18 00:08:01
  description  : character(0)
  heading      : character(0)
  id           : p2009a.txt
  language     : en
  origin       : character(0)

In [23]:
# 말뭉치 중 특정문서의 메타정보(meta)확인
my[[13]]$meta

  author       : character(0)
  datetimestamp: 2019-01-18 00:08:01
  description  : character(0)
  heading      : character(0)
  id           : p2014c.txt
  language     : en
  origin       : character(0)

In [24]:
# 말뭉치 중 특정문서의 메타정보(meta) 업데이트
meta(my[[1]], tag = 'author') <- '백영민'
meta(my[[1]], tag = 'description') <- '퍼블리싱된 논문의 초록'

my[[1]]$meta

  author       : 백영민
  datetimestamp: 2019-01-18 00:08:01
  description  : 퍼블리싱된 논문의 초록
  heading      : character(0)
  id           : p2009a.txt
  language     : en
  origin       : character(0)

In [25]:
# 말뭉치 중 특정문서의 메타정보(meta) 업데이트
meta(my[[13]], tag = 'author') <- '백영민'
meta(my[[13]], tag = 'description') <- '초록내용에 8개 항목(4개 소주제와 4개 설명문구)으로 구성되어 있음'

my[[13]]$meta

  author       : 백영민
  datetimestamp: 2019-01-18 00:08:01
  description  : 초록내용에 8개 항목(4개 소주제와 4개 설명문구)으로 구성되어 있음
  heading      : character(0)
  id           : p2014c.txt
  language     : en
  origin       : character(0)

### 말뭉치 전체 내용조회

In [26]:
# 말뭉치 각 문서별 내용조회
sapply(my, 'extract', 1) %>% print %>% class
# - sapply(my, '[', 1) 동일결과

# - my라는 말뭉치를 구성하는 각 문서들이 리스트구조로 연결되어 있으며, 
#   각 문서단위 리스트항목별로 가지고 있는 1: content, 2: meta 세부요소항목 중 
#   1: content 부분을 데이터순환방식으로 로딩한 것임
# ==> 추출결과는 리스트객체인데, 각 문서 내용이 리스트항목의 개별요소로 되어 있어,
#     전처리 작업시 바로 접근하기 용이함

$p2009a.txt.content
[1] "Does deliberative setting, online versus face-to-face, influence citizens' experiences? Are certain factors differently influential in one setting than in the other? We draw on a nationally representative survey and identify  citizens who participated in both online and face-to-face settings (n  = 82). We use structural equation modeling to first assess the effects that deliberation format has on the interrelationship between such crucial factors as motivations to deliberate, perceived diversity, elicited emotions, enhanced understanding, and goal evaluation. We later employ network analysis to ask which factor or which cluster of factors is more central to an overall experience in which format. Relying on citizens who participated in both settings and using within-subject analyses, we  assure that the previously unnoted findings are attributable to the format per se rather than to individual characteristics. We discuss the theoretical, practical and methodolog

[1] "list"

In [27]:
# 말뭉치 각 문서별 내용조회
lapply(my, 'extract', 1) %>% print %>% class
# - lapply(my, '[', 1) 동일결과

# - my라는 말뭉치를 구성하는 각 문서들이 리스트구조로 연결되어 있으며, 
#   각 문서단위 리스트항목별로 가지고 있는 1: content, 2: meta 세부요소항목 중 
#   1: content 부분을 데이터순환방식으로 로딩한 것임
# ==> 추출결과는 리스트객체인데, 각 문서 내용이 리스트항목의 "세부" 요소로 되어 있어,
#     리스트구조가 이중레벨이라 전처리 작업시 접근하기 불편함

$p2009a.txt
$p2009a.txt$content
[1] "Does deliberative setting, online versus face-to-face, influence citizens' experiences? Are certain factors differently influential in one setting than in the other? We draw on a nationally representative survey and identify  citizens who participated in both online and face-to-face settings (n  = 82). We use structural equation modeling to first assess the effects that deliberation format has on the interrelationship between such crucial factors as motivations to deliberate, perceived diversity, elicited emotions, enhanced understanding, and goal evaluation. We later employ network analysis to ask which factor or which cluster of factors is more central to an overall experience in which format. Relying on citizens who participated in both settings and using within-subject analyses, we  assure that the previously unnoted findings are attributable to the format per se rather than to individual characteristics. We discuss the theoretical, practical an

[1] "list"

### 말뭉치 전체 메타정보 조회

In [28]:
# 말뭉치 각 문서별 메타정보 조회
sapply(my, extract, 2) %>% print %>% class
# - lapply(my, '[', 2) 동일결과

# - my라는 말뭉치를 구성하는 각 문서들이 리스트구조로 연결되어 있으며, 
#   각 문서단위 리스트항목별로 가지고 있는 1: content, 2: meta 세부요소항목 중 
#   2: meta 부분을 데이터순환방식으로 로딩한 것임
# ==> 추출결과는 리스트객체인데, 각 문서 메타정보가 리스트항목의 개별요소로 되어 있어,
#     전처리 작업시 바로 접근하기 용이함

$p2009a.txt.meta
  author       : 백영민
  datetimestamp: 2019-01-18 00:08:01
  description  : 퍼블리싱된 논문의 초록
  heading      : character(0)
  id           : p2009a.txt
  language     : en
  origin       : character(0)

$p2009b.txt.meta
  author       : character(0)
  datetimestamp: 2019-01-18 00:08:01
  description  : character(0)
  heading      : character(0)
  id           : p2009b.txt
  language     : en
  origin       : character(0)

$p2010a.txt.meta
  author       : character(0)
  datetimestamp: 2019-01-18 00:08:01
  description  : character(0)
  heading      : character(0)
  id           : p2010a.txt
  language     : en
  origin       : character(0)

$p2010b.txt.meta
  author       : character(0)
  datetimestamp: 2019-01-18 00:08:01
  description  : character(0)
  heading      : character(0)
  id           : p2010b.txt
  language     : en
  origin       : character(0)

$p2010c.txt.meta
  author       : character(0)
  datetimestamp: 2019-01-18 00:08:01
  description  : character(0)
  h

[1] "list"

In [29]:
# 말뭉치 각 문서별 메타정보 조회
lapply(my, extract, 2) %>% print %>% class
# - lapply(my, '[', 2) 동일결과

# - my라는 말뭉치를 구성하는 각 문서들이 리스트구조로 연결되어 있으며, 
#   각 문서단위 리스트항목별로 가지고 있는 1: content, 2: meta 세부요소항목 중 
#   2: meta 부분을 데이터순환방식으로 로딩한 것임
# ==> 추출결과는 리스트객체인데, 각 문서의 메타정보가 리스트항목의 "세부" 요소로 되어 있어,
#     리스트구조가 이중레벨이라 전처리 작업시 접근하기 불편함

$p2009a.txt
$p2009a.txt$meta
  author       : 백영민
  datetimestamp: 2019-01-18 00:08:01
  description  : 퍼블리싱된 논문의 초록
  heading      : character(0)
  id           : p2009a.txt
  language     : en
  origin       : character(0)


$p2009b.txt
$p2009b.txt$meta
  author       : character(0)
  datetimestamp: 2019-01-18 00:08:01
  description  : character(0)
  heading      : character(0)
  id           : p2009b.txt
  language     : en
  origin       : character(0)


$p2010a.txt
$p2010a.txt$meta
  author       : character(0)
  datetimestamp: 2019-01-18 00:08:01
  description  : character(0)
  heading      : character(0)
  id           : p2010a.txt
  language     : en
  origin       : character(0)


$p2010b.txt
$p2010b.txt$meta
  author       : character(0)
  datetimestamp: 2019-01-18 00:08:01
  description  : character(0)
  heading      : character(0)
  id           : p2010b.txt
  language     : en
  origin       : character(0)


$p2010c.txt
$p2010c.txt$meta
  author       : character(0)
  date

[1] "list"

# <font color = 'blue'>말뭉치 텍스트셋 전처리 (p.117)</font>

## tm패키지의 전용 전처리함수 이용
* tm패키지의 <font color = 'red'>tm_map(말뭉치객체, <font color = 'blue'>전용전처리함수</font>)</font> 함수를 이용해 
<br>데이터순환 방식으로 분석대상 텍스트셋의 세부단위별로 필요한 전처리 작업을 진행함
* 단, tm패키지의 전처리관련 함수를 사용하기 위해서는 대상객체가 코퍼스(말뭉치) 객체유형이어야하며, 
<br> 이를 통해 전처리된 결과객체도 코퍼스(말뭉치) 객체형식을 유지하게 됨
<pre></pre> 

* <font color = 'red'>tm_map(<font color = 'black'>말뭉치객체</font>, <font color = 'blue'>removeNumbers</font>)</font>: 말뭉치에 사용된 숫자표현을 모두 제거함
* <font color = 'red'>tm_map(<font color = 'black'>말뭉치객체</font>, <font color = 'blue'>removePunctuation</font>)</font>: 말뭉치에 사용된 문장부호 및 특수문자를 모드 제거함
* <font color = 'red'>tm_map(<font color = 'black'>말뭉치객체</font>, <font color = 'blue'>removeWords</font>)</font>: 말뭉치에 사용된 미리지정한 단어(불용어)를 삭제함
* <font color = 'red'>tm_map(<font color = 'black'>말뭉치객체</font>, <font color = 'blue'>stripWhitespace</font>)</font> 말뭉치에 사용된 2개 이상 연이어 등장하는 공백을 1개의 공백으로 변환함
* <font color = 'red'>tm_map(<font color = 'black'>말뭉치객체</font>, <font color = 'blue'>stemDocument</font>)</font> 말뭉치에 어근 동일화 알고리즘을 적용함

<pre></pre> 
* tm패키지의 다양한 전처리 함수를 무조건적으로 적용하게 되면, 
<br>관련된 문구들이 일괄처리 되면서 중요한 고유명사나 표현들도 같이 삭제될 수 있으므로 
<br>대상패턴이나 적용결과 등을 확인해 가면서 사용해야함 

## tm패키지에 일반함수를 이용한 전처리
* tm패키지의 <font color = 'red'>tm_map(말뭉치객체, content_transfer(일반함수))</font> 함수를 이용해 
<br>데이터순환 방식으로 분석대상 텍스트셋의 세부단위별로 필요한 전처리 작업을 진행함
* 단, tm패키지에 전처리관련 일반 함수를 사용하기 위해서는 대상객체가 코퍼스(말뭉치) 객체유형이어야하며, 
<br> 이를 통해 전처리된 결과객체도 코퍼스(말뭉치) 객체형식을 유지하게 됨
<pre></pre> 

* <font color = 'red'>tm_map(<font color = 'black'>말뭉치객체</font>, <font color = 'blue'>content_transfer</font>(<font color = 'green'>일반함수</font>))</font>: 필요한 R 내외부함수/사용자정의 함수를 별도로 적용함

## stringr 패키지를 이용한 전처리
* 문자열 전용 패키지인 stringr패키지의 str_ 시리즈 함수를 통해서 텍스트데이터에 대한 전처리 작업을 수행함
* str_ 시리즈 함수를 사용해 전처리를 진행하면, 원래 입력한 코퍼스(말뭉치) 객체형식이 사라지고 일반 리스트 객체형식으로 변형됨

# <font color = 'blue'>말뭉치 문서별 내용(content) 개수 통일 (p.117)</font>

## 말뭉치 문서 중 내용(content)항목의 세부요소 개수가 차이가 나는 부분만을 개별로 조정
* 말뭉치에 들어 있는 문서들마다 세부요소 개수가 차이가 날때 
<br>해당 문서들을 하나씩 로딩해 세부요소를 통합하고 다시 해당문서파일 내용에 일일이 업데이트 하는 작업실시

### 세부요소 개수가 차이나는 개별문서 특성과 내용조회

In [30]:
# 세부요소 개수가 차이가나는 문서파일의 내용(content)와 메타정보(meta)를 별도로 추출함
tmp <- my[[13]]
print(tmp)

<<PlainTextDocument>>
Metadata:  7
Content:  chars: 1386


In [31]:
# 세부요소 개수가 차이가나는 문서파일의 내용(content)부분만을 별도의 임시변수에 저장함
tmp_content <- tmp$content
tmp_content %>% print

length(tmp_content) %>% print
class(tmp_content) %>% print

[1] "BACKGROUND AND AIMS:"                                                                                                                                                                                                                                                                                                                                                                                                 
[2] "Little is known about the extent depression adds to the costs of treatment for physical health conditions. This study examined the paths and the extent to which depression in conjunction with a physical health problem is associated with an increase in healthcare expenditures and how that is different between younger and older adults."                                                                      
[3] "METHODS:"                                                                                                                                                                  

### 해당 개별문서의 세부요소를 하나로 통합함

In [32]:
# stringr::str_c()함수를 이용해 세부요소 개수가 여러개로 구성된 문서파일의 내용(content)을 하나로 통합해줌
# - base::paste(tmp_content, collapse = '') 동일결과

tmp_combined <- str_c(tmp_content, collapse = '')
tmp_combined %>% print

length(tmp_combined) %>% print
class(tmp_combined) %>% print

[1] "BACKGROUND AND AIMS:Little is known about the extent depression adds to the costs of treatment for physical health conditions. This study examined the paths and the extent to which depression in conjunction with a physical health problem is associated with an increase in healthcare expenditures and how that is different between younger and older adults.METHODS:Data from the 2007 Medical Expenditure Panel Survey (MEPS) were analyzed. Depression status and physical health conditions were identified through ICD-9 codes. The multiple group structural equation modeling (SEM) was employed to examine the moderated mediation effects.RESULTS:Approximately 11% of adults had clinical depression. The multiple group SEM for both younger and older adult groups supports not only a direct effect of depression on expenditures but also an indirect effect via comorbid health conditions. Furthermore, the indirect effect was significantly more prominent among older respondents than among younger ones,

### 여러개의 세부요소를 하나로 통합한 내용(content)를 해당문서파일 content항목에 업데이트

In [33]:
# 하나의 요소로 통합된 내용을 해당 문서파일에 업데이트 실시
my[[13]]$content <- tmp_combined

# 해당문서파일의 내용(content)부분을 다시 조회해 여러개가 아닌 한 개요소로 통합된 내용인지 확인
my[[13]]$content %>% print %>% length %>% print

[1] "BACKGROUND AND AIMS:Little is known about the extent depression adds to the costs of treatment for physical health conditions. This study examined the paths and the extent to which depression in conjunction with a physical health problem is associated with an increase in healthcare expenditures and how that is different between younger and older adults.METHODS:Data from the 2007 Medical Expenditure Panel Survey (MEPS) were analyzed. Depression status and physical health conditions were identified through ICD-9 codes. The multiple group structural equation modeling (SEM) was employed to examine the moderated mediation effects.RESULTS:Approximately 11% of adults had clinical depression. The multiple group SEM for both younger and older adult groups supports not only a direct effect of depression on expenditures but also an indirect effect via comorbid health conditions. Furthermore, the indirect effect was significantly more prominent among older respondents than among younger ones,

## 말뭉치 문서들의 내용(content)항목의 세부요소 개수를 동일하게 만들기
* 말뭉치에 들어 있는 문서들마다 세부요소 개수가 차이가 날때 
<br>해당 문서들을 하나씩 로딩해 세부요소를 통합해 다시 일일이 업데이트 하는 작업은 비효율적임
* 말뭉치 문서들의 내용을 일률적으로 로딩해 각각 한 개 내용으로 통합한 다음 
<br>일률적으로 말뭉치에 업데이트 하는 데이터순환방식을 이용하는 것이 효과적임

### 원본 텍스트셋 파일들로부터 다시 말뭉치를 재생성해서 테스트 진행

In [34]:
# 말뭉치 생성대상 텍스트데이터 위치경로 설정
loc_papers_en <- './datatm/ymbaek_papers_en'

# tm패키지 Vcorpus() 함수를 이용해 텍스트셋 파일들을 통합해 말뭉치로 재생성함
my <-VCorpus(DirSource(loc_papers_en))

### 말뭉치 기본 특성파악

In [35]:
# 말뭉치 간단출력
my %>% print
# - Metadata: 말뭉치를 구성하는 각 파일에 대한 별도의 설명은 없는 상태임
# - Content: 24개 문서(documents)로 구성되어 있음을 알 수 있음

sprintf('말뭉치 객체유형:')
class(my) %>% print
# - Corpus객체도 리스트 객체구조를 가지고 있는데, 
#   결합되는 여러개 텍스트셋 파일별 내용과 메타정보를 독립적으로 각 리스트항목으로 유지하는 구조임

sprintf('말뭉치 구성요소 길이: %d', length(my))  %>% print

<<VCorpus>>
Metadata:  corpus specific: 0, document level (indexed): 0
Content:  documents: 24


[1] "말뭉치 객체유형:"

[1] "VCorpus" "Corpus" 
[1] "말뭉치 구성요소 길이: 24"


In [36]:
# 말뭉치 객체의 각 문서별 content 항목의 세부요소갯수 파악 
sapply(my, lengths) %>% print %>% class
# - 말뭉치를 구성하고 있는 각 리스트항목(문서)에 데이터순환방식으로 접근해 
#   content 항목의 길이를 카운팅하고, 결과를 행렬구조로 보여줌
# - 대부분 문서파일의 content항목의 길이는 1로 나와서 1개의 문자열로 구성되어 있지만,
#   p2014c.txt 파일에 들어 있는 content항목의 길이는 8개로 파악되고 있어
#   이를 unlist를 적용해 1개 문자열로 만들어야 함

        p2009a.txt p2009b.txt p2010a.txt p2010b.txt p2010c.txt p2011a.txt
content          1          1          1          1          1          1
meta             7          7          7          7          7          7
        p2011b.txt p2012a.txt p2012b.txt p2013a.txt p2014a.txt p2014b.txt
content          1          1          1          1          1          1
meta             7          7          7          7          7          7
        p2014c.txt p2014d.txt p2014e.txt p2014f.txt p2014g.txt p2014h.txt
content          8          1          1          1          1          1
meta             7          7          7          7          7          7
        p2014i.txt p2014k.txt p2015a.txt p2015b.txt p2015c.txt p2015d.txt
content          1          1          1          1          1          1
meta             7          7          7          7          7          7


[1] "matrix"

### 말뭉치 각 문서파일의 내용(content)를 한 개 요소로 통일화

In [37]:
# 말뭉치 각 문서파일의 내용을 일괄 로딩
tmp_all <- sapply(my, 'extract', 1) 
tmp_all  %>% print

# - 대부분 문서파일의 content항목의 내용은 1개 문자열로 되어 있는데, 
#   p2014c.txt 파일에 들어 있는 content항목의 내용은 8개로 파악됨

length(tmp_all_combined) %>% print
class(tmp_all_combined) %>% print

$p2009a.txt.content
[1] "Does deliberative setting, online versus face-to-face, influence citizens' experiences? Are certain factors differently influential in one setting than in the other? We draw on a nationally representative survey and identify  citizens who participated in both online and face-to-face settings (n  = 82). We use structural equation modeling to first assess the effects that deliberation format has on the interrelationship between such crucial factors as motivations to deliberate, perceived diversity, elicited emotions, enhanced understanding, and goal evaluation. We later employ network analysis to ask which factor or which cluster of factors is more central to an overall experience in which format. Relying on citizens who participated in both settings and using within-subject analyses, we  assure that the previously unnoted findings are attributable to the format per se rather than to individual characteristics. We discuss the theoretical, practical and methodolog

ERROR: Error in eval(lhs, parent, parent): 객체 'tmp_all_combined'를 찾을 수 없습니다


In [38]:
# 각 문서파일마다 content 내용을 한 개 요소로 통일화 작업실시
tmp_all_combined <- sapply(tmp_all, str_c, collapse = '') 
tmp_all_combined %>% print
# - 각 문서파일마다 content 내용이 1개 문자열로 일원화 되었음

length(tmp_all_combined) %>% print
class(tmp_all_combined) %>% print

[1] 24
[1] "character"


### 각 문서별 세부요소를 하나씩으로 통일화한 내용(content)를 해당문서파일의 content항목마다 업데이트

In [39]:
# 각 문서별로 하나의 요소로 통합된 내용을 해당 문서파일 content 항목에 개별 업데이트 실시

# for loop 방식을 이용한 업데이트
for (i in 1:length(tmp_all_combined)) {
    my[[i]]$content <- tmp_all_combined[i]
}

In [40]:
# 말뭉치 객체의 각 문서별 업데이트된 content 항목의 세부요소갯수 파악 
sapply(my, lengths) %>% print %>% class

# - 말뭉치를 구성하고 있는 각 리스트항목(문서)에 데이터순환방식으로 접근해 
#   content 항목의 길이를 카운팅하고, 결과를 행렬구조로 보여줌
# - 모든 문서파일의 content항목의 길이는 1로 나와서 1개의 문자열로 구성되어 있는 것으로 확인됨

        p2009a.txt p2009b.txt p2010a.txt p2010b.txt p2010c.txt p2011a.txt
content          1          1          1          1          1          1
meta             7          7          7          7          7          7
        p2011b.txt p2012a.txt p2012b.txt p2013a.txt p2014a.txt p2014b.txt
content          1          1          1          1          1          1
meta             7          7          7          7          7          7
        p2014c.txt p2014d.txt p2014e.txt p2014f.txt p2014g.txt p2014h.txt
content          1          1          1          1          1          1
meta             7          7          7          7          7          7
        p2014i.txt p2014k.txt p2015a.txt p2015b.txt p2015c.txt p2015d.txt
content          1          1          1          1          1          1
meta             7          7          7          7          7          7


[1] "matrix"

# <font color = 'blue'>공백 처리(stripping white space) (p.125)</font>
* 공백(white space)은 단어와 단어를 구분하는 기능을 수행
* 문자 입력의 실수, 또는 온라인 텍스트 수집과정의 오류로 공백이 2개 이상 발견되는데, 이를 1개로 축약시키는 과정임

## 검색패턴을 이용한 텍스트셋 탐색

### 공백 검색패턴 설정

In [41]:
# 문서내 공백이 2개이상 들어간 문구를 찾는 정규표현식
src <- '[[:graph:]]{0,}[[:space:]]{2,}[[:graph:]]{0,}'

# 가운데 [[:space:]]{2,} ==> 공백문자에 해당하는 요소가 2개 이상 들어 있는 패턴검색
# 양쪽   [[:graph:]]{0,} ==> 공백문자 2개이상 있는 패턴 전후로 어떠한 문자가 오는지(없던지, 1개 이상 오던지) 패턴검색

### 공백 검색패턴을 통한 텍스트셋 탐색

#### 각 문서별 검색패턴 일치내용 파악

In [42]:
sprintf('말뭉치 문서별 공백2개이상 패턴 포함여부를 content항목에서 출현내용으로 파악:')
sapply(my, str_extract_all, pattern = src) %>% print
# - sapply(my, gregexpr, pattern = src) %>% regmatche s(x = my) %>% print 동일결과
# - 검색패턴에 해당하는 공백2개이상 표현이 해당 텍스트요소에 들어 있지 않으면 character(0)을 출력함
# - 앞서 각 문서별 세부요소를 하나씩으로 통일화 전처리한 my 코퍼스객체를 투입함
# - 말뭉치를 구성하는 각 문서별로 검색패턴과 일치하는 공백2개이상 출현 실제내용을 파악함

[1] "말뭉치 문서별 공백2개이상 패턴 포함여부를 content항목에서 출현내용으로 파악:"

$p2009a.txt
[1] "identify  citizens" "(n  ="              "we  assure"        

$p2009b.txt
character(0)

$p2010a.txt
character(0)

$p2010b.txt
character(0)

$p2010c.txt
[1] "consumerism  has"      "engagement,  others"   "that:  (1)"           
[4] "different;  (2)"       "political   consumers"

$p2011a.txt
character(0)

$p2011b.txt
character(0)

$p2012a.txt
[1] "two.  Drawing"                 "we   examine"                 
[3] "  these"                       "deliberation  over-represents"
[5] "moderates,    generates"       "action.     At"               

$p2012b.txt
character(0)

$p2013a.txt
character(0)

$p2014a.txt
character(0)

$p2014b.txt
[1] "communication   (CVC)" "  framework"           "like  :),"            
[4] "like  ^_^."            "hypotheses  in"       

$p2014c.txt
character(0)

$p2014d.txt
character(0)

$p2014e.txt
character(0)

$p2014f.txt
character(0)

$p2014g.txt
character(0)

$p2014h.txt
[1] "Abstract   This"             "\"self\"   (operationalized"
[3] "se

#### 말뭉치 전체내용 중 검색패턴 일치내용 빈도수 파악

In [43]:
sprintf('전체 말뭉치에서 공백2개이상 패턴 출현내용을 문자순서 기준 정렬:')
sapply(my, str_extract_all, pattern = src) %>% 
    unlist %>% table %>% print %>% sum
# - 말뭉치를 구성하는 각 문서가 아닌 전체 내용을 통합했을 때 
#   검색패턴과 일치하는 공백2개이상의 실제내용을 문자순서를 기준으로 정렬해 파악함
# - 공백2개이상 패턴이 포함된 경우의 문구는 30개로 나타남 

[1] "전체 말뭉치에서 공백2개이상 패턴 출현내용을 문자순서 기준 정렬:"

.
                    framework                         these 
                            1                             1 
    "self"   (operationalized                         (n  = 
                            1                             1 
              Abstract   This                action.     At 
                            1                             1 
         avoidance.     Anger         communication   (CVC) 
                            1                             1 
             consumerism  has deliberation  over-represents 
                            1                             1 
              different;  (2)                  discussed.   
                            1                             1 
          engagement,  others               enhanced  among 
                            1                             1 
               hypotheses  in            identify  citizens 
                            1                             1 
         levels.  Theo

[1] 30

In [44]:
sprintf('전체 말뭉치에서 공백2개이상 패턴 출현내용을 빈도수 기준 내림차순으로 정렬:')
sapply(my, str_extract_all, pattern = src) %>% 
    unlist %>% table %>% sort(decreasing = TRUE) %>% print %>% sum
# - 말뭉치를 구성하는 각 문서가 아닌 전체 내용을 통합했을 때 
#   검색패턴과 일치하는 실제 공백2개이상 내용의 출현빈도수를 기준으로 내림차순으로 정렬해 파악함

[1] "전체 말뭉치에서 공백2개이상 패턴 출현내용을 빈도수 기준 내림차순으로 정렬:"

.
                    framework                         these 
                            1                             1 
    "self"   (operationalized                         (n  = 
                            1                             1 
              Abstract   This                action.     At 
                            1                             1 
         avoidance.     Anger         communication   (CVC) 
                            1                             1 
             consumerism  has deliberation  over-represents 
                            1                             1 
              different;  (2)                  discussed.   
                            1                             1 
          engagement,  others               enhanced  among 
                            1                             1 
               hypotheses  in            identify  citizens 
                            1                             1 
         levels.  Theo

[1] 30

#### 각 문서별 검색패턴 일치내용을 데이터프레임 구조로 파악

In [45]:
# 전체 말뭉치에서 공백2개이상 패턴 출현내용을 별도의 임시변수에 저장
my_pp_tb <- sapply(my, str_extract_all, pattern = src) %>% unlist %>% table
my_pp_tb

.
                    framework                         these 
                            1                             1 
    "self"   (operationalized                         (n  = 
                            1                             1 
              Abstract   This                action.     At 
                            1                             1 
         avoidance.     Anger         communication   (CVC) 
                            1                             1 
             consumerism  has deliberation  over-represents 
                            1                             1 
              different;  (2)                  discussed.   
                            1                             1 
          engagement,  others               enhanced  among 
                            1                             1 
               hypotheses  in            identify  citizens 
                            1                             1 
         levels.  Theo

In [46]:
# 전체 말뭉치에서 공백2개이상 출현내용을 별도의 데이터프레임 객체로 저장
my_pp_df <- data.frame(term = names(my_pp_tb), freq = c(my_pp_tb), row.names = NULL)
my_pp_df %>% print

                            term freq
1                      framework    1
2                          these    1
3      "self"   (operationalized    1
4                          (n  =    1
5                Abstract   This    1
6                 action.     At    1
7           avoidance.     Anger    1
8          communication   (CVC)    1
9               consumerism  has    1
10 deliberation  over-represents    1
11               different;  (2)    1
12                  discussed.      1
13           engagement,  others    1
14               enhanced  among    1
15                hypotheses  in    1
16            identify  citizens    1
17          levels.  Theoretical    1
18                     like  :),    1
19                    like  ^_^.    1
20                manifested   a    1
21       moderates,    generates    1
22        participation.   Using    1
23         political   consumers    1
24        potential  limitations    1
25       self-esteem    compared    1
26          

In [47]:
# 전체 말뭉치에서 공백2개이상 출현내용을 빈도수를 기준으로 내림차순 정렬
my_pp_df %>% dplyr::arrange(desc(freq), term) %>% print

                            term freq
1                      framework    1
2                          these    1
3      "self"   (operationalized    1
4                          (n  =    1
5                Abstract   This    1
6                 action.     At    1
7           avoidance.     Anger    1
8          communication   (CVC)    1
9               consumerism  has    1
10 deliberation  over-represents    1
11               different;  (2)    1
12                  discussed.      1
13           engagement,  others    1
14               enhanced  among    1
15                hypotheses  in    1
16            identify  citizens    1
17          levels.  Theoretical    1
18                     like  :),    1
19                    like  ^_^.    1
20                manifested   a    1
21       moderates,    generates    1
22        participation.   Using    1
23         political   consumers    1
24        potential  limitations    1
25       self-esteem    compared    1
26          

In [48]:
# 전체 말뭉치에서 공백2개이상 출현내용을 일정수준 이상의 빈도수를 기준으로 내림차순 정렬
my_pp_df %>% arrange(desc(freq), term) %>% filter(freq >= 5) %>% print
# - 5회 이상 반복적으로 나타나는 공백2개이상 문구는 없는 것으로 나타남

[1] term freq
<0 rows> (or 0-length row.names)


## 텍스트셋 탐색결과에 따른 전처리 수행
* 각 문서별로, 또는 말뭉치 전체적으로 출현한 2개이상 공백에 대한 내림차순 정렬결과, 특별한 의미를 가지는 공백은 없는 것으로 나타남
* 이에 따라 말뭉치내 각 문서별로 포함되어 있는 공백 2개연속이상 나오는 패턴들을 일괄 삭제하는 것으로 전처리 방향성을 정리함

### tm패키지 전용 전처리함수 이용

In [49]:
# tm::tm_map()에 전용 전처리함수 옵션을 이용한 전처리 
# - 이 결과로 코퍼스(말뭉치)객체형식이 유지되어 
#   tm패키지의 전용 전처리함수 옵션적용을 추가적으로 계속할 수 있음

my_pp_stws <- tm_map(my, stripWhitespace)
# - pp: pre-processing
# - stws: strip whitespace
# - 앞서 전처리된 my 코퍼스객체를 전처리 대상객체로 입력함
# - 코퍼스 각 문서별로 포함되어 있는 공백2개이상 표현들을 일괄적으로 1칸짜리 공백으로 변경함

class(my_pp_stws)
# - tm패키지 tm_map()에 전용 전처리함수 옵션인 stripWhitespace을 이용했으므로
#   전처리결과도 코퍼스(말뭉치) 객체형식을 유지하고 있음

[1] "VCorpus" "Corpus"

In [50]:
# 전처리 대상이 제대로 처리되었는지 출현내용 기술통계파악
sprintf('tm패키지를 이용한 공백2개이상 표현 전처리 결과:')
sapply(my_pp_stws, str_extract_all, pattern = '[[:graph:]]*[[:space:]]{2,}[[:graph:]]*') %>% 
    unlist %>% table %>% sort(decreasing = TRUE) %>% print

# - 말뭉치를 구성하는 각 문서가 아닌 전체 내용을 통합했을 때 
#   검색패턴과 일치하는 공백2개이상 출현 실제내용의 빈도수를 내림차순으로 파악한 결과
#   각 문서별 모든 공백2개이상 표현들이 1칸까리 공백으로 일괄 변경되었음을 알 수 있음

[1] "tm패키지를 이용한 공백2개이상 표현 전처리 결과:"

integer(0)


### tm패키지에 일반함수를 이용한 전처리

In [51]:
# tm::tm_map()에 속해 있는 전용 전처리 함수가 아니라 일반 전처리 함수를 이용한 방식 
# - 이 결과로 코퍼스(말뭉치)객체형식이 유지되어 
#   tm패키지 전처리함수 적용을 추가적으로 계속할 수 있음

my_pp_erws <- tm_map(my, content_transformer(str_replace_all), pattern = '[[:space:]]{2,}', replacement = ' ')
# - erws: erase strip whitespace 전처리
# - 앞서 전처리된 my 코퍼스객체를 전처리 대상객체로 입력함
# - 코퍼스 각 문서별로 포함되어 있는 공백2개이상 표현들을 일괄적으로 1칸짜리 공백으로 변경함
# - tm_map()에 전용전처리 함수옵션이 아닌 content_transformer()를 이용해 
#   일반 내/외부/사용자정의함수를 전처리 함수로 사용함

# (주의!!!)
# pattern = [[:graph:]]*[[:space:]]{2,}[[:graph:]]* 이 옵션으로 전처리를 하면 문제발생함
# ==> 2개이상 연속으로 발생한 공백과 그 전후로 있는 문자코드를 다같이 변경하므로 원본데이터 내용 일부가 사라지게 됨
# pattern = [[:space:]]{2,} ==> 2개이상 연속으로 발생한 공백만을 탐색해 공백1칸으로 전처리할 수 있도록 함

class(my_pp_erws)
# - tm패키지 tm_map()에 일반 내외부/사용자정의함수를 사용할 수 있는 content_transformer을 이용했으므로
#   전처리결과도 코퍼스(말뭉치) 객체형식을 유지하고 있음

[1] "VCorpus" "Corpus"

In [52]:
# 전처리 대상이 제대로 처리되었는지 출현내용 기술통계파악
sprintf('stringr패키지를 이용한 2개이상 공백 전처리 결과:')
sapply(my_pp_erws, str_extract_all, pattern = '[[:graph:]]*[[:space:]]{2,}[[:graph:]]*') %>% 
    unlist %>% table %>% sort(decreasing = TRUE) %>% print

# - 말뭉치를 구성하는 각 문서가 아닌 전체 내용을 통합해 
#   검색패턴과 일치하는 2개이상 공백이 발생한 부분들의 빈도수를 내림차순으로 파악한 결과
#   각 문서별 모든 2개이상 공백들이 일괄적으로 공백1칸으로 변경되었음을 알 수 있음

[1] "stringr패키지를 이용한 2개이상 공백 전처리 결과:"

integer(0)


### stringr패키지를 이용한 전처리

In [53]:
# stringr::str_replace_all() 함수를 이용한 전처리
# - 이 결과로 코퍼스(말뭉치)객체가 문자열객체로 변환되어, 
#   tm패키지 전처리함수 적용을 하려면 다시 말뭉치객체로 만드는 작업이 필요함

my_pp_delws <- sapply(my, str_replace_all, pattern = '[[:space:]]{2,}', replacement = ' ')
# - delws: delete whitespace
# - 코퍼스 각 문서별로 포함되어 있는 2개이상 연이어 발생한 공백들을 일괄적으로 1칸짜리 공백으로 변경함

# (주의!!!)
# pattern = [[:graph:]]*[[:space:]]{2,}[[:graph:]]* 이 옵션으로 전처리를 하면 문제발생함
# ==> 2개이상 연속으로 발생한 공백과 그 전후로 있는 문자코드를 다같이 변경하므로 원본데이터 내용 일부가 사라지게 됨
# pattern = [[:space:]]{2,} ==> 2개이상 연속으로 발생한 공백만을 탐색해 공백1칸으로 전처리할 수 있도록 함

class(my_pp_delws)
# - stringr패키지의 문자열 전처리 함수인 str_replace_all을 이용했으므로 
#   전처리결과도 코퍼스(말뭉치) 객체형식이 사라지고 일반 리스트 객체형식으로 변형됨

[1] "character"

In [54]:
# 전처리 대상이 제대로 처리되었는지 출현내용 기술통계파악
sprintf('stringr패키지를 이용한 2개이상 공백 전처리 결과:')
sapply(my_pp_delws, str_extract_all, pattern = '[[:graph:]]*[[:space:]]{2,}[[:graph:]]*') %>% 
    unlist %>% table %>% sort(decreasing = TRUE) %>% print

# - 말뭉치를 구성하는 각 문서가 아닌 전체 내용을 통합해 
#   검색패턴과 일치하는 2개이상 공백이 발생한 부분들의 빈도수를 내림차순으로 파악한 결과
#   각 문서별 모든 2개이상 공백들이 일괄적으로 공백1칸으로 변경되었음을 알 수 있음

[1] "stringr패키지를 이용한 2개이상 공백 전처리 결과:"

integer(0)


# <font color = 'blue'>숫자표현(numeric data) 제거 (p.123)</font>
* 텍스트셋에는 양적인 크기나 서열/순서의 의미, 범주의 의미, 변화의 의미로서 숫자를 사용하므로 이에 대한 적절한 처리가 필요함
* 일률적인 숫자텍스트 삭제 이외에, 여건에 따라 특정한 수치적 표현은 고유명사로 유지해야하는 경우도 있음

## 검색패턴을 이용한 텍스트셋 탐색

### 숫자표현 검색패턴 설정

In [55]:
# 문서내 숫자가 들어간 문구를 찾는 정규표현식
src <- '(-?)([[:digit:]]{1,})(.?)([[:digit:]]{0,})'

# (-?) ==> 음수를 의미하는 마이너스(-)표시가 없던지/1개 나타나는 패턴탐색
# [[:digit:]]{1,} ==> 숫자가 1개 이상 나타나는 패턴탐색 
# .? ==> 소숫점을 의미하는 점(.)표시가 없던지/1개 나타나는 패턴탐색
# [[:digit:]]{0,} ==> 소수점 다음에 소수 숫자가 없던지 1개 이상 존재하는 패턴탐색

### 숫자표현 검색패턴을 통한 텍스트셋 탐색

#### 각 문서별 검색패턴 일치내용 파악

In [56]:
sprintf('말뭉치 문서별 숫자패턴 포함여부를 content항목에서 출현내용으로 파악:')
sapply(my_pp_stws, str_extract_all, pattern = src) %>% print
# - sapply(my_pp_stws, gregexpr, pattern = src) %>% regmatches(x = my_pp_stws) %>% print 동일결과
# - 앞서 공백2개이상 문구를 1개로 전처리한 my_pp_stws 코퍼스객체을 인풋으로 사용
# - 말뭉치를 구성하는 각 문서별로 검색패턴과 일치하는 숫자출현 실제내용을 파악함
# - 검색패턴에 해당하는 숫자표현이 해당 텍스트요소에 들어 있지 않으면 character(0)을 출력함

[1] "말뭉치 문서별 숫자패턴 포함여부를 content항목에서 출현내용으로 파악:"

$p2009a.txt
[1] "82)"

$p2009b.txt
[1] "2004 " "35 "  

$p2010a.txt
[1] "756)"

$p2010b.txt
[1] "1)" "2)" "3)" "4)"

$p2010c.txt
[1] "2002 " "1)"    "2)"    "3)"   

$p2011a.txt
[1] "2008 "

$p2011b.txt
[1] "2003)"

$p2012a.txt
character(0)

$p2012b.txt
[1] "1)"    "2)"    "2008 "

$p2013a.txt
character(0)

$p2014a.txt
[1] "2028)" "1)"    "2)"    "3)"   

$p2014b.txt
[1] "78 "

$p2014c.txt
[1] "2007 " "-9 "   "11%"  

$p2014d.txt
[1] "1)" "2)" "3)"

$p2014e.txt
[1] "1)" "2)" "3)" "4)"

$p2014f.txt
character(0)

$p2014g.txt
[1] "2012 "

$p2014h.txt
[1] "712 "

$p2014i.txt
[1] "1973."

$p2014k.txt
[1] "2012 " "1)"    "2)"    "3)"   

$p2015a.txt
[1] "1 " "2 " "3 " "1 " "2 "

$p2015b.txt
character(0)

$p2015c.txt
character(0)

$p2015d.txt
character(0)



#### 말뭉치 전체내용 중 검색패턴 일치내용 빈도수 파악

In [57]:
sprintf('전체 말뭉치에서 숫자패턴 출현내용을 문자순서 기준 정렬:')
sapply(my_pp_stws, str_extract_all, pattern = src) %>% 
    unlist %>% table %>% print
# - 말뭉치를 구성하는 각 문서가 아닌 전체 내용을 통합했을 때 
#   검색패턴과 일치하는 숫자들의 실제내용을 문자순서를 기준으로 정렬해 파악함

[1] "전체 말뭉치에서 숫자패턴 출현내용을 문자순서 기준 정렬:"

.
  -9     1     1)   11% 1973.    2     2) 2002  2003) 2004  2007  2008  2012  
    1     2     7     1     1     2     7     1     1     1     1     2     2 
2028)    3     3)   35     4)  712   756)   78    82) 
    1     1     6     1     2     1     1     1     1 


In [58]:
sprintf('전체 말뭉치에서 숫자패턴 출현내용을 빈도수 기준 내림차순으로 정렬:')
sapply(my_pp_stws, str_extract_all, pattern = src) %>% 
    unlist %>% table %>% sort(decreasing = TRUE) %>% print
# - 말뭉치를 구성하는 각 문서가 아닌 전체 내용을 통합했을 때 
#   검색패턴과 일치하는 실제 숫자내용의 출현빈도수를 기준으로 내림차순으로 정렬해 파악함

[1] "전체 말뭉치에서 숫자패턴 출현내용을 빈도수 기준 내림차순으로 정렬:"

.
   1)    2)    3)    1     2  2008  2012     4)   -9    11% 1973. 2002  2003) 
    7     7     6     2     2     2     2     2     1     1     1     1     1 
2004  2007  2028)    3    35   712   756)   78    82) 
    1     1     1     1     1     1     1     1     1 


#### 각 문서별 검색패턴 일치내용을 데이터프레임 구조로 파악

In [59]:
# 전체 말뭉치에서 숫자패턴 출현내용을 별도의 임시변수에 저장
my_pp_tb <- sapply(my_pp_stws, str_extract_all, pattern = src) %>% unlist %>% table
my_pp_tb

.
  -9     1     1)   11% 1973.    2     2) 2002  2003) 2004  2007  2008  2012  
    1     2     7     1     1     2     7     1     1     1     1     2     2 
2028)    3     3)   35     4)  712   756)   78    82) 
    1     1     6     1     2     1     1     1     1 

In [60]:
# 전체 말뭉치에서 숫자패턴 출현내용을 별도의 데이터프레임 객체로 저장
my_pp_df <- data.frame(term = names(my_pp_tb), freq = c(my_pp_tb), row.names = NULL)
my_pp_df %>% print

    term freq
1    -9     1
2     1     2
3     1)    7
4    11%    1
5  1973.    1
6     2     2
7     2)    7
8  2002     1
9  2003)    1
10 2004     1
11 2007     1
12 2008     2
13 2012     2
14 2028)    1
15    3     1
16    3)    6
17   35     1
18    4)    2
19  712     1
20  756)    1
21   78     1
22   82)    1


In [61]:
# 전체 말뭉치에서 숫자패턴 출현내용을 빈도수를 기준으로 내림차순 정렬
my_pp_df %>% dplyr::arrange(desc(freq), term) %>% print

    term freq
1     1)    7
2     2)    7
3     3)    6
4     1     2
5     2     2
6  2008     2
7  2012     2
8     4)    2
9    -9     1
10   11%    1
11 1973.    1
12 2002     1
13 2003)    1
14 2004     1
15 2007     1
16 2028)    1
17    3     1
18   35     1
19  712     1
20  756)    1
21   78     1
22   82)    1


In [62]:
# 전체 말뭉치에서 숫자패턴 출현내용을 일정수준 이상의 빈도수를 기준으로 내림차순 정렬
my_pp_df %>% arrange(desc(freq), term) %>% filter(freq >= 5) %>% print

  term freq
1   1)    7
2   2)    7
3   3)    6


## 텍스트셋 탐색결과에 따른 전처리 수행
* 각 문서별로, 또는 말뭉치 전체적으로 출현한 숫자들에 대한 내림차순 정렬결과, 특별한 의미를 가지는 숫자가 없는 것으로 나타남
* 이에 따라 말뭉치내 각 문서별로 포함되어 있는 숫자표현들은 일괄 삭제하는 것으로 전처리 방향성을 정리함

### tm패키지 전용 전처리함수 이용

In [63]:
# tm::tm_map()에 전용 전처리함수 옵션을 이용한 전처리 
# - 이 결과로 코퍼스(말뭉치)객체형식이 유지되어 
#   tm패키지의 전용 전처리함수 옵션적용을 추가적으로 계속할 수 있음

my_pp_rmnum <- tm_map(my_pp_stws, removeNumbers)
# - rmnum: remove numeric character
# - 앞서 전처리된 공백2개이상 문구를 1개로 변경한 my_pp_stws 코퍼스객체를 전처리 대상객체로 입력함
# - 코퍼스 각 문서별로 포함되어 있는 숫자표현들을 일괄적으로 삭제함

class(my_pp_rmnum)
# - tm패키지 tm_map()에 전용 전처리함수 옵션인 removeNumbers를 이용했으므로
#   전처리결과도 코퍼스(말뭉치) 객체형식을 유지하고 있음

[1] "VCorpus" "Corpus"

In [64]:
# 전처리 대상이 제대로 처리되었는지 출현내용 기술통계파악
sprintf('tm패키지를 이용한 숫자표현 전처리 결과:')
sapply(my_pp_rmnum, str_extract_all, pattern = '-?[[:digit:]]{1,}.?[[:digit:]]{0,}') %>% 
    unlist %>% table %>% sort(decreasing = TRUE) %>% print

# - 말뭉치를 구성하는 각 문서가 아닌 전체 내용을 통합했을 때 
#   검색패턴과 일치하는 숫자출현 실제내용의 빈도수를 내림차순으로 파악한 결과
#   각 문서별 모든 숫자표현들이 일괄 삭제되었음을 알 수 있음

[1] "tm패키지를 이용한 숫자표현 전처리 결과:"

integer(0)


### tm패키지에 일반함수를 이용한 전처리

In [65]:
# tm::tm_map()에 속해 있는 전용 전처리 함수가 아니라 일반 전처리 함수를 이용한 방식 
# - 이 결과로 코퍼스(말뭉치)객체형식이 유지되어 
#   tm패키지 전처리함수 적용을 추가적으로 계속할 수 있음

my_pp_ernum <- tm_map(my_pp_erws, content_transformer(str_replace_all), 
                      pattern = '-?[[:digit:]]{1,}.?[[:digit:]]{0,}', replacement = ' ')
# - ernum: erase numeric data 전처리
# - 앞서 전처리된 공백2개이상 문구를 삭제한 my_pp_erws 코퍼스(말뭉치)객체를 전처리 대상객체로 입력함
# - 코퍼스 각 문서별로 포함되어 있는 숫자표현들을 일괄적으로 공백으로 변경함

class(my_pp_ernum)
# - tm패키지 tm_map()에 일반 내외부/사용자정의함수를 사용할 수 있는 content_transformer을 이용했으므로
#   전처리결과도 코퍼스(말뭉치) 객체형식을 유지하고 있음

[1] "VCorpus" "Corpus"

In [66]:
# 전처리 대상이 제대로 처리되었는지 출현내용 기술통계파악
sprintf('stringr패키지를 이용한 숫자표현 전처리 결과:')
sapply(my_pp_ernum, str_extract_all, pattern = '-?[[:digit:]]{1,}.?[[:digit:]]{0,}') %>% 
    unlist %>% table %>% sort(decreasing = TRUE) %>% print

# - 말뭉치를 구성하는 각 문서가 아닌 전체 내용을 통합했을 때 
#   검색패턴과 일치하는 숫자출현 실제내용의 빈도수를 내림차순으로 파악한 결과
#   각 문서별 모든 숫자표현들이 일괄 삭제되었음을 알 수 있음

[1] "stringr패키지를 이용한 숫자표현 전처리 결과:"

integer(0)


### stringr패키지를 이용한 전처리

In [67]:
# stringr::str_replace_all() 함수를 이용한 전처리
# - 이 결과로 코퍼스(말뭉치)객체가 문자열객체로 변환되어, 
#   tm패키지 전처리함수 적용을 하려면 다시 말뭉치객체로 만드는 작업이 필요함

my_pp_delnum <- sapply(my_pp_delws, str_replace_all, pattern = '-?[[:digit:]]{1,}.?[[:digit:]]{0,}', replacement = ' ')
# - delnum: delete numeric character
# - 앞서 전처리된 공백2개이상 문구를 삭제한 my_pp_delws 일반문자열 객체를 전처리 대상객체로 입력함
# - 코퍼스 각 문서별로 포함되어 있는 숫자표현들을 일괄적으로 삭제함

class(my_pp_delnum)
# - stringr패키지의 문자열 전처리 함수인 str_replace_all을 이용했으므로 
#   전처리결과도 코퍼스(말뭉치) 객체형식이 사라지고 일반 리스트 객체형식으로 변형됨

[1] "character"

In [68]:
# 전처리 대상이 제대로 처리되었는지 출현내용 기술통계파악
sprintf('stringr패키지를 이용한 숫자표현 전처리 결과:')
sapply(my_pp_delnum, str_extract_all, pattern = '-?[[:digit:]]{1,}.?[[:digit:]]{0,}') %>% 
    unlist %>% table %>% sort(decreasing = TRUE) %>% print

# - 말뭉치를 구성하는 각 문서가 아닌 전체 내용을 통합했을 때 
#   검색패턴과 일치하는 숫자출현 실제내용의 빈도수를 내림차순으로 파악한 결과
#   각 문서별 모든 숫자표현들이 일괄 삭제되었음을 알 수 있음

[1] "stringr패키지를 이용한 숫자표현 전처리 결과:"

integer(0)


# <font color = 'blue'>대소문자 통일(Case-insensitive) (p.125)</font>
* 영문 텍스트셋은 대소문자를 구별해서 사용하므로, 이에 대한 적절한 처리가 필요함
* 일률적인 소문자로 통일하는 방법 이외에, 여건에 따라 특정한 문구는 대문자나 대소문자 혼용을 유지해야하는 경우도 있음

## 검색패턴을 이용한 텍스트셋 탐색

### 대문자 검색패턴 설정

In [69]:
# 문서내 대문자가 들어간 문구를 찾는 정규표현식
src <- '[[:graph:]]*[[:upper:]]{1,}[[:graph:]]*'

# 가운데 [[:upper:]]{1,} ==> 대문자에 해당하는 요소가 1개 이상 들어 있는 패턴검색
# 양쪽   [[:graph:]]* ==> 대문자 1개이상 있는 패턴 전후로 어떠한 문자가 오는지 패턴검색

### 대문자 검색패턴을 통한 텍스트셋 탐색

#### 각 문서별 검색패턴 일치내용 파악

In [70]:
sprintf('말뭉치 문서별 대문자패턴 포함여부를 content항목에서 출현내용으로 파악:')
sapply(my_pp_rmnum, str_extract_all, pattern = src) %>% print
# - sapply(my, gregexpr, pattern = src) %>% regmatche s(x = my) %>% print 동일결과
# - 검색패턴에 해당하는 대문자표현이 해당 텍스트요소에 들어 있지 않으면 character(0)을 출력함
# - 앞서 (공백2칸을 1칸으로) + (숫자표현을 일괄삭제한) 전처리한 my_pp_rmnum 코퍼스객체를 투입함
# - 말뭉치를 구성하는 각 문서별로 검색패턴과 일치하는 대문자출현 실제내용을 파악함

[1] "말뭉치 문서별 대문자패턴 포함여부를 content항목에서 출현내용으로 파악:"

$p2009a.txt
[1] "Does"    "Are"     "We"      "We"      "We"      "Relying" "We"     

$p2009b.txt
 [1] "The"       "We"        "The"       "(IRT)"     "Drawing"   "National" 
 [7] "Annenberg" "Election"  "Study,"    "Using"     "We"       

$p2010a.txt
[1] "We"       "American" "(N"       "We"       "We"       "We"       "Weak"    
[8] "Our"     

$p2010b.txt
[1] "Ambivalence" "In"          "This"        "A"           "Some"       
[6] "The"        

$p2010c.txt
[1] "Political"   "While"       "Drawing"     "Findings"    "Theoretical"

$p2011a.txt
 [1] "Party"         "However,"      "Black"         "Democratic"   
 [5] "White"         "Democrats"     "Focusing"      "Black"        
 [9] "White"         "Relying"       "National"      "Presidential" 
[13] "Election"      "Barack"        "Obama"         "John"         
[17] "McCain"        "White"         "Black"         "Black"        
[21] "White"         "Democrats"     "Independents," "White"        
[25] "Democrats,"    "White"   

#### 말뭉치 전체내용 중 검색패턴 일치내용 빈도수 파악

In [71]:
sprintf('전체 말뭉치에서 대문자패턴 출현내용을 문자순서 기준 정렬:')
sapply(my_pp_rmnum, str_extract_all, pattern = src) %>% 
    unlist %>% table %>% print
# - 말뭉치를 구성하는 각 문서가 아닌 전체 내용을 통합했을 때 
#   검색패턴과 일치하는 대문자들의 실제내용을 문자순서를 기준으로 정렬해 파악함

[1] "전체 말뭉치에서 대문자패턴 출현내용을 문자순서 기준 정렬:"

.
                        (CVC)                         (IRT) 
                            1                             1 
                       (ISPs)                        (MEPS) 
                            1                             1 
                           (N                         (SEM) 
                            2                             2 
                        (SNA)                         (SNS) 
                            1                             3 
                       (SNSs)                       (SNSs), 
                            1                             1 
                            A                      Abstract 
                            1                             1 
                      Action,                 Additionally, 
                            1                             1 
          adults.METHODS:Data                     Affective 
                            1                             3 
                  Affi

In [72]:
sprintf('전체 말뭉치에서 대문자패턴 출현내용을 빈도수 기준 내림차순으로 정렬:')
sapply(my_pp_rmnum, str_extract_all, pattern = src) %>% 
    unlist %>% table %>% sort(decreasing = TRUE) %>% print
# - 말뭉치를 구성하는 각 문서가 아닌 전체 내용을 통합했을 때 
#   검색패턴과 일치하는 실제 대문자내용의 출현빈도수를 기준으로 내림차순으로 정렬해 파악함

[1] "전체 말뭉치에서 대문자패턴 출현내용을 빈도수 기준 내림차순으로 정렬:"

.
                          SNS                        Korean 
                           19                            13 
                           We                          This 
                           13                            12 
                          The                         White 
                            9                             6 
                        Based                            By 
                            5                             5 
                      Drawing                       Relying 
                            5                             5 
                        South                   Theoretical 
                            5                             5 
                        Black                  Implications 
                            4                             4 
                          Our                         Using 
                            4                             4 
                      

#### 각 문서별 검색패턴 일치내용을 데이터프레임 구조로 파악

In [73]:
# 전체 말뭉치에서 대문자패턴 출현내용을 별도의 임시변수에 저장
my_pp_tb <- sapply(my_pp_rmnum, str_extract_all, pattern = src) %>% unlist %>% table
my_pp_tb

.
                        (CVC)                         (IRT) 
                            1                             1 
                       (ISPs)                        (MEPS) 
                            1                             1 
                           (N                         (SEM) 
                            2                             2 
                        (SNA)                         (SNS) 
                            1                             3 
                       (SNSs)                       (SNSs), 
                            1                             1 
                            A                      Abstract 
                            1                             1 
                      Action,                 Additionally, 
                            1                             1 
          adults.METHODS:Data                     Affective 
                            1                             3 
                  Affi

In [74]:
# 전체 말뭉치에서 대문자패턴 출현내용을 별도의 데이터프레임 객체로 저장
my_pp_df <- data.frame(term = names(my_pp_tb), freq = c(my_pp_tb), row.names = NULL)
my_pp_df %>% print

                             term freq
1                           (CVC)    1
2                           (IRT)    1
3                          (ISPs)    1
4                          (MEPS)    1
5                              (N    2
6                           (SEM)    2
7                           (SNA)    1
8                           (SNS)    3
9                          (SNSs)    1
10                        (SNSs),    1
11                              A    1
12                       Abstract    1
13                        Action,    1
14                  Additionally,    1
15            adults.METHODS:Data    1
16                      Affective    3
17                    Affirmative    1
18                        African    1
19     age.CONCLUSIONS:Depression    1
20                    AIMS:Little    1
21                       Although    2
22                    Ambivalence    1
23                       American    1
24                      Americans    2
25                       

In [75]:
# 전체 말뭉치에서 대문자패턴 출현내용을 빈도수를 기준으로 내림차순 정렬
my_pp_df %>% dplyr::arrange(desc(freq), term) %>% print

                             term freq
1                             SNS   19
2                          Korean   13
3                              We   13
4                            This   12
5                             The    9
6                           White    6
7                           Based    5
8                              By    5
9                         Drawing    5
10                        Relying    5
11                          South    5
12                    Theoretical    5
13                          Black    4
14                   Implications    4
15                            Our    4
16                          Using    4
17                          (SNS)    3
18                      Affective    3
19                       Findings    3
20                       Internet    3
21                          K-pop    3
22                        Results    3
23                           SNSs    3
24                         SNSs’    3
25                        

In [76]:
# 전체 말뭉치에서 대문자패턴 출현내용을 일정수준 이상의 빈도수를 기준으로 내림차순 정렬
my_pp_df %>% arrange(desc(freq), term) %>% filter(freq >= 5) %>% print

          term freq
1          SNS   19
2       Korean   13
3           We   13
4         This   12
5          The    9
6        White    6
7        Based    5
8           By    5
9      Drawing    5
10     Relying    5
11       South    5
12 Theoretical    5


## 텍스트셋 탐색결과에 따른 전처리 수행
* 각 문서별로, 또는 말뭉치 전체적으로 출현한 대문자 포함 문구에 대한 내림차순 정렬결과, 특별한 의미를 가지는 대문자문구는 없는 것으로 나타남
* 이에 따라 말뭉치내 각 문서별로 포함되어 있는 대문자 패턴들을 일괄 소문자로 변환하는 것으로 전처리 방향성을 정리함

### tm패키지에 일반함수를 이용한 전처리

In [77]:
# tm::tm_map()에 속해 있는 전용 전처리 함수가 아니라 일반 전처리 함수를 이용한 방식 
# - 이 결과로 코퍼스(말뭉치)객체형식이 유지되어 
#   tm패키지 전처리함수 적용을 추가적으로 계속할 수 있음

my_pp_tolw <- tm_map(my_pp_rmnum, content_transformer(tolower))
# - tolw: to lower alphabet
# - 앞서 전처리된 숫자표현 문구를 삭제한 my_pp_rmnum 코퍼스객체를 전처리 대상객체로 입력함
# - 코퍼스 각 문서별로 포함되어 있는 대문자를 일괄적으로 소문자로 변환함

# - tm_map() 함수를 이용한 전처리 작업시에 자체적으로 제공되는 전처리옵션 이외에 
#   R의 다양한 내장/외부함수/사용자정의함수를 통한 전처리를 하려면
#   tm_map() 함수 안에 content_fransformer(내외부/사용자정의 함수) 함수를 옵션으로 사용하면 됨

class(my_pp_tolw)
# - tm패키지 tm_map()에 일반 함수를 사용가능하도록 해주는 content_transformer를 이용했으므로
#   전처리결과도 코퍼스(말뭉치) 객체형식을 유지하고 있음

[1] "VCorpus" "Corpus"

In [78]:
# 전처리 대상이 제대로 처리되었는지 출현내용 기술통계파악
sprintf('tm패키지를 이용한 대문자 일괄 소문자로 전처리 결과:')
sapply(my_pp_tolw, str_extract_all, pattern = '[[:graph:]]*[[:upper:]]{1,}[[:graph:]]*') %>% 
    unlist %>% table %>% sort(decreasing = TRUE) %>% print

# - 말뭉치를 구성하는 각 문서가 아닌 전체 내용을 통합해 
#   검색패턴과 일치하는 대문자 문구의 빈도수를 내림차순으로 파악한 결과
#   각 문서별 대문자 문구들이 일괄 소문자로 변경되었음을 알 수 있음

[1] "tm패키지를 이용한 대문자 일괄 소문자로 전처리 결과:"

integer(0)


### stringr패키지를 이용한 전처리

In [79]:
# stringr::str_replace_all() 함수를 이용한 전처리
# - 이 결과로 코퍼스(말뭉치)객체가 문자열객체로 변환되어, 
#   tm패키지 전처리함수 적용을 하려면 다시 말뭉치객체로 만드는 작업이 필요함

my_pp_tosmall <- sapply(my_pp_delnum, str_replace_all, pattern = '[[:upper:]]{1,}', replacement = tolower)
# - tomini: to small alphabet
# - 앞서 전처리된 숫자표현 문구를 삭제한 my_pp_delnum 코퍼스객체를 전처리 대상객체로 입력함
# - tolower함수를 이용해 코퍼스 각 문서별로 포함되어 있는 대문자들을 일괄적으로 소문자로 변경함

# (주의!!!)
# pattern = '[[:graph:]]*[[:upper:]]{1,}[[:graph:]]* 이 옵션으로 전처리를 하면 문제발생함
# ==> 대문자가 사용된 문구와 그 전후로 있는 문자코드를 다같이 소문자 처리하므로 원본데이터에 영향을 미침
# pattern = [[:upper:]]{1,} ==> 대문자가 사용된 부문만을 탐색해 전처리할 수 있도록 함

class(my_pp_tosmall)
# - stringr패키지의 문자열 전처리 함수인 str_replace_all을 이용했으므로 
#   전처리결과도 코퍼스(말뭉치) 객체형식이 사라지고 일반 리스트 객체형식으로 변형됨

[1] "character"

In [80]:
# 전처리 대상이 제대로 처리되었는지 출현내용 기술통계파악
sprintf('stringr패키지를 이용한 대문자 일괄 소문자로 전처리 결과:')
sapply(my_pp_tosmall, str_extract_all, pattern = '[[:graph:]]*[[:upper:]]{1,}[[:graph:]]*') %>% 
    unlist %>% table %>% sort(decreasing = TRUE) %>% print

# - 말뭉치를 구성하는 각 문서가 아닌 전체 내용을 통합해 
#   검색패턴과 일치하는 대문자 문구의 빈도수를 내림차순으로 파악한 결과
#   각 문서별 대문자 문구들이 일괄 소문자로 변경되었음을 알 수 있음

[1] "stringr패키지를 이용한 대문자 일괄 소문자로 전처리 결과:"

integer(0)


# <font color = 'blue'>문장부호(punctuation marks)와 특수문자(special characters) 제거 (p.123)</font>
* 텍스트셋에는 많은 문장부호들이 사용되며, 각 문장부호는 고유한 문법적인 기능과 의미를 담고 있음
* 문장부호인 마침표(.), 콤마(,), 콜론(:), 세미콜론(;), 특수문자인 !@#$%^&*()-_+=/<> 등을 상황에 따라 제거필요

## 검색패턴을 이용한 텍스트셋 탐색

### 문장부호&특수문자 검색패턴 설정

In [81]:
src <- '[[:graph:]]{0,}[[:punct:]]{1,}[[:graph:]]{0,}'

# 가운데 [[:punct:]]{1,} ==> 문장부호&특수문자에 해당하는 요소가 1개 이상 들어 있는 패턴검색 ==> [[:punct:]]+ 동일검색패턴
# 양쪽   [[:graph:]]{0,} ==> 문장부호&특수문자가 1개이상 있는 패턴 전후로 어떠한 문자가 오는지 패턴검색 ==> [[:graph:]]* 동일검색패턴

### 문장부호&특수문자 검색패턴을 통한 텍스트셋 탐색

#### 각 문서별 검색패턴 일치내용 파악

In [82]:
sprintf('말뭉치 문서별 문장부호&특수문자 패턴 포함여부를 content항목에서 출현내용으로 파악:')
sapply(my_pp_tolw, str_extract_all, pattern = src) %>% print
# - sapply(my, gregexpr, pattern = src) %>% regmatche s(x = my) %>% print 동일결과
# - 앞서 (공백2칸을 1칸으로) + (숫자표현을 일괄삭제) + (대소문자를 통일) 전처리된 my_pp_tolw 코퍼스객체를 사용함
# - 말뭉치를 구성하는 각 문서별로 검색패턴과 일치하는 문장부호&특수문자 출현 실제내용을 파악함
# - 검색패턴에 해당하는 문장부호&특수문자 표현이 해당 텍스트요소에 들어 있지 않으면 character(0)을 출력함

[1] "말뭉치 문서별 문장부호&특수문자 패턴 포함여부를 content항목에서 출현내용으로 파악:"

$p2009a.txt
 [1] "setting,"         "face-to-face,"    "citizens'"        "experiences?"    
 [5] "other?"           "face-to-face"     "(n"               ")."              
 [9] "deliberate,"      "diversity,"       "emotions,"        "understanding,"  
[13] "evaluation."      "format."          "within-subject"   "analyses,"       
[17] "characteristics." "theoretical,"     "implications."   

$p2009b.txt
 [1] "inconclusive,"    "knowledge,"       "citizenry."       "generally,"      
 [5] "knowledge."       "(irt)"            "characteristics," "difficulty,"     
 [9] "response."        "study,"           "comedy."          "meta-analysis"   
[13] "items,"           "knowledge,"       "options,"         "citizens."       
[17] "effects."        

$p2010a.txt
 [1] "face-to-face"   "(n"             ")."             "follow-up"     
 [5] "strong,"        "weak,"          "ideologues."    "-"             
 [9] "-"              "follow-up"      "engagement,"    "strength."     
[13] "fol

#### 말뭉치 전체내용 중 검색패턴 일치내용 빈도수 파악

In [83]:
sprintf('전체 말뭉치에서 문장부호&특수문자 패턴 출현내용을 문자순서 기준 정렬:')
sapply(my_pp_tolw, str_extract_all, pattern = src) %>% 
    unlist %>% table %>% print %>% sum
# - 말뭉치를 구성하는 각 문서가 아닌 전체 내용을 통합했을 때 
#   검색패턴과 일치하는 문장부호&특수문자들의 실제내용을 문자순서를 기준으로 정렬해 파악함

[1] "전체 말뭉치에서 문장부호&특수문자 패턴 출현내용을 문자순서 기준 정렬:"

.
                            -                      "others" 
                            2                             1 
               "self-esteem")                        "self" 
                            1                             1 
                      "social                             % 
                            1                             1 
                           ()                           (), 
                           22                             1 
                          (a)                           (b) 
                            2                             2 
                          (c)                         (cvc) 
                            1                             1 
                       (e.g.,                        (i.e., 
                            3                             5 
                        (irt)                        (isps) 
                            1                             1 
                      

[1] 489

In [84]:
sprintf('전체 말뭉치에서 문장부호&특수문자 패턴 출현내용을 빈도수 기준 내림차순으로 정렬:')
sapply(my_pp_tolw, str_extract_all, pattern = src) %>% 
    unlist %>% table %>% sort(decreasing = TRUE) %>% print %>% sum
# - 말뭉치를 구성하는 각 문서가 아닌 전체 내용을 통합했을 때 
#   검색패턴과 일치하는 실제 문장부호&특수문자 내용의 출현빈도수를 기준으로 내림차순으로 정렬해 파악함

[1] "전체 말뭉치에서 문장부호&특수문자 패턴 출현내용을 빈도수 기준 내림차순으로 정렬:"

.
                           ()                    discussed. 
                           22                             9 
               cross-cultural                        study, 
                            6                             6 
                       (i.e.,                  face-to-face 
                            5                             5 
                      users’                     findings, 
                            5                             4 
                     however,                participation. 
                            4                             4 
                       (e.g.,                            (n 
                            3                             3 
                        (sns)                    addiction. 
                            3                             3 
                    citizens'                   conditions. 
                            3                             3 
                    emo

[1] 489

#### 각 문서별 검색패턴 일치내용을 데이터프레임 구조로 파악

In [85]:
# 전체 말뭉치에서 문장부호&특수문자 패턴 출현내용을 별도의 임시변수에 저장
my_pp_tb <- sapply(my_pp_tolw, str_extract_all, pattern = src) %>% unlist %>% table
my_pp_tb

.
                            -                      "others" 
                            2                             1 
               "self-esteem")                        "self" 
                            1                             1 
                      "social                             % 
                            1                             1 
                           ()                           (), 
                           22                             1 
                          (a)                           (b) 
                            2                             2 
                          (c)                         (cvc) 
                            1                             1 
                       (e.g.,                        (i.e., 
                            3                             5 
                        (irt)                        (isps) 
                            1                             1 
                      

In [86]:
# 전체 말뭉치에서 문장부호&특수문자 패턴 출현내용을 별도의 데이터프레임 객체로 저장
my_pp_df <- data.frame(term = names(my_pp_tb), freq = c(my_pp_tb), row.names = NULL)
my_pp_df %>% print %>% nrow

                             term freq
1                               -    2
2                        "others"    1
3                  "self-esteem")    1
4                          "self"    1
5                         "social    1
6                               %    1
7                              ()   22
8                             (),    1
9                             (a)    2
10                            (b)    2
11                            (c)    1
12                          (cvc)    1
13                         (e.g.,    3
14                         (i.e.,    5
15                          (irt)    1
16                         (isps)    1
17                         (meps)    1
18                             (n    3
19               (operationalized    2
20                          (sem)    2
21                          (sna)    1
22                          (sns)    3
23                         (snss)    1
24                        (snss),    1
25                       

[1] 353

In [87]:
# 전체 말뭉치에서 문장부호&특수문자 패턴 출현내용을 빈도수를 기준으로 내림차순 정렬
my_pp_df %>% dplyr::arrange(desc(freq), term) %>% print

                             term freq
1                              ()   22
2                      discussed.    9
3                  cross-cultural    6
4                          study,    6
5                          (i.e.,    5
6                    face-to-face    5
7                         users’    5
8                       findings,    4
9                        however,    4
10                 participation.    4
11                         (e.g.,    3
12                             (n    3
13                          (sns)    3
14                     addiction.    3
15                      citizens'    3
16                    conditions.    3
17                      emotions,    3
18                      follow-up    3
19                          k-pop    3
20                     open-ended    3
21                       people's    3
22                       privacy,    3
23                    skepticism,    3
24                         snss’    3
25                         

In [88]:
# 전체 말뭉치에서 문장부호&특수문자 패턴 출현내용을 일정수준 이상의 빈도수를 기준으로 내림차순 정렬
my_pp_df %>% arrange(desc(freq), term) %>% filter(freq >= 5) %>% print

            term freq
1             ()   22
2     discussed.    9
3 cross-cultural    6
4         study,    6
5         (i.e.,    5
6   face-to-face    5
7        users’    5


## 텍스트셋 탐색결과에 따른 전처리 방향성 설정
* 문장부호&특수기호를 일률적으로 삭제하기 보다는, 특정 문장부호&특수기호가 그냥 삭제되었을 때 의미해석이 어려운 경우를 감안하여 
<br>문장부호&특수기호 중 중요한 패턴들에 대해서 세부적으로 전처리를 진행함 
<pre></pre>
* 문구에 <font color = 'red'>하이픈(-)</font> 표시가 있는 경우
<br>- 하이픈기호를 중심으로 연결된 단어를 분리하고, 일괄적으로 하이픈기호 제거
* 문구에 <font color = 'red'>슬래시(/)</font> 표시가 있는 경우
<br>- 슬래시기호를 중심으로 연결된 단어의 독립성이 인정되므로 슬래시기호를 공백으로 처리하여 두 개 단어로 분리하도록 함
* 문구에 <font color = 'red'>어포스트로피(단어's, 단어’s)</font> 표시가 있는 경우
<br>- 특별히 어포스트로피 표현자체가 중요한 고유명사로서의 의미가 있는 경우는 없는 것으로 파악되어 어포스트로피 표현만을 일괄 삭제하는 것으로 처리하기로 함
* 문구에 <font color = 'red'>점물음표(.?)</font> 표시가 있는 경우
<br>- 점물음표 표현자체가 중요한 고유명사로서의 의미가 있는 경우는 없는 것으로 파악되어 점물음표 문구 표현만을 일괄 삭제하여 전후로 연결된 단어을 독립적으로 분리하는 것으로 전처리함
* 문구에 <font color = 'red'>(알파벳1개)</font> 표시가 있는 경우: (a), (b), (c)
<br>- 괄호사이에 영어철자가 1개 오는 문구는 중요한 고유명사로서의 의미가 있는 경우는 없는 것으로 파악되어 일괄삭제처리함
* 문구에 <font color = 'red'>특정문구</font>를 다른 문구로 1:1로 직접변경
<br>- 특정한 문구에 대한 전처리시에 정규표현식 사용이 복잡하고, 해당 패턴도 적은 경우에는 1:1 직접변환방식으로 상황에 따라 변경/삭제함
* 문구에 <font color = 'red'>남은 문장부호와 특수문자</font> 일괄제거
<br>- 남아 있는 의미 없는 문장부호&특수기호는 일괄삭제하고, 단어는 단어대로 남겨지도록 처리함

## 문구에 하이픈(-) 표시가 있는 경우

### 문구에 하이픈(-) 표시가 있는 패턴탐색

In [89]:
# 전체 말뭉치에서 문장부호&특수문자 패턴 중 특정한 출현내용을 별도로 확인
my_pp_df %>% arrange(desc(freq), term) %>% 
    filter(str_detect(term, pattern = '[[:graph:]]{0,}-[[:graph:]]{0,}')) %>% print

# 정규표현식
# - 양쪽 [[:graph:]]{0,} ==> 하이픈 기호 양쪽에 연결되어 있는 문구확인 ==> [[:graph:]]* 동일검색패턴

# 전처리 방향
# - 하이픈기호가 포함된 문구는 연결된 단어들이 서로결합되어 하나의 표현을 구성하는 경우로 간주해 
#   하이픈을 없애고 하나의 단어로 만들도록 함
# - 하이픈기호가 포함된 문구 중에서도 별도로 문장부호가 포함되어 있는 경우에는
#   일단 하이픈기호를 중심으로 연결된 단어를 분리하고나서, 일괄적으로 하이픈기호를 제거하는 작업을 실시하기로 함

                     term freq
1          cross-cultural    6
2            face-to-face    5
3               follow-up    3
4                   k-pop    3
5              open-ended    3
6                       -    2
7             large-scale    2
8        mediation-effect    2
9      privacy-protective    2
10      self-presentation    2
11         "self-esteem")    1
12             ‘opt-in’    1
13            ‘opt-out’    1
14 “notice-and-consent”    1
15    anxious-ambivalent.    1
16            blue-collar    1
17          co-activation    1
18          co-constraint    1
19           co-emergence    1
20           co-existence    1
21        content-analyze    1
22      cross-culturally.    1
23            cross-party    1
24       culture-specific    1
25            data-driven    1
26   dismissive-avoidant,    1
27           eye-oriented    1
28          face-to-face,    1
29      fearful-avoidant,    1
30       five-dimensional    1
31                   icd-    1
32     inter-d

### tm패키지에 일반함수를 이용한 전처리

In [90]:
# tm::tm_map()에 속해 있는 전용 전처리 함수가 아니라 일반 전처리 함수를 이용한 방식 
# - 이 결과로 코퍼스(말뭉치)객체형식이 유지되어 
#   tm패키지 전처리함수 적용을 추가적으로 계속할 수 있음

my_pp_pmsc <- tm_map(my_pp_tolw, content_transformer(str_replace_all), 
                     pattern = '-', replacement = '')
# - pmsc: punctuation marks & special character
# - 앞서 (공백2칸을 1칸으로) + (숫자표현을 일괄삭제) + (대소문자를 통일) 전처리된 my_pp_tolw 코퍼스객체를 사용함
# - 코퍼스 각 문서별로 포함되어 있는 하이픈기호는 삭제하고, 전후로 연결된 단어를 하나의 결합단어로 변경해줌

# (주의!!!)
# pattern = '[[:graph:]]{0,}-[[:graph:]]{0,}' 이 옵션으로 전처리를 하면 문제발생함
# ==> 하이픈과 그 전후로 있는 문구를 다같이 삭제하므로 원본데이터에 영향을 미침
# pattern = '-' ==> 하이픈이 사용된 부문만을 탐색해 전처리할 수 있도록 함

class(my_pp_pmsc)
# - tm패키지 tm_map()에 일반 함수를 사용가능하도록 해주는 content_transformer를 이용했으므로
#   전처리결과도 코퍼스(말뭉치) 객체형식을 유지하고 있음

# 하이픈기호가 사용된 문구는 전처리되어 해당되는 사항이 없음
sapply(my_pp_pmsc , str_extract_all, pattern = '[[:graph:]]{0,}-[[:graph:]]{0,}')  %>% 
    unlist %>% table %>% sort(decreasing = TRUE) %>% print

[1] "VCorpus" "Corpus"

integer(0)


In [91]:
# 하이픈기호가 사용된 문구는 전처리되어 해당되는 사항이 없음

sprintf('전체 말뭉치에서 문장부호&특수문자 패턴 출현내용을 빈도수 기준 내림차순으로 정렬:')
my_pp_tb <- sapply(my_pp_pmsc, str_extract_all, pattern = '[[:graph:]]{0,}[[:punct:]]{1,}[[:graph:]]{0,}') %>% 
    unlist %>% table %>% sort(decreasing = TRUE) 
my_pp_df <- data.frame(term = names(my_pp_tb), freq = c(my_pp_tb), row.names = NULL)
my_pp_df %>% dplyr::arrange(desc(freq), term) %>% print

[1] "전체 말뭉치에서 문장부호&특수문자 패턴 출현내용을 빈도수 기준 내림차순으로 정렬:"

                             term freq
1                              ()   22
2                      discussed.    9
3                          study,    6
4                          (i.e.,    5
5                         users’    5
6                       findings,    4
7                        however,    4
8                  participation.    4
9                          (e.g.,    3
10                             (n    3
11                          (sns)    3
12                     addiction.    3
13                      citizens'    3
14                    conditions.    3
15                      emotions,    3
16                       people's    3
17                       privacy,    3
18                    skepticism,    3
19                         snss’    3
20                            (a)    2
21                            (b)    2
22               (operationalized    2
23                          (sem)    2
24                             ).    2
25                    activ

### stringr패키지를 이용한 전처리

In [92]:
# stringr::str_replace_all() 함수를 이용한 전처리
# - 이 결과로 코퍼스(말뭉치)객체가 문자열객체로 변환되어, 
#   tm패키지 전처리함수 적용을 하려면 다시 말뭉치객체로 만드는 작업이 필요함

my_pp_pmnsc <- sapply(my_pp_tolw, str_replace_all, pattern = '-', replacement = '')
# - pmsc: punctuation marks and special character
# - 앞서 (공백2칸을 1칸으로) + (숫자표현을 일괄삭제) + (대소문자를 통일) 전처리된 my_pp_tolw 코퍼스객체를 사용함
# - 코퍼스 각 문서별로 포함되어 있는 하이픈기호는 삭제하고, 전후로 연결된 단어를 하나의 결합단어로 변경해줌

# (주의!!!)
# pattern = '[[:graph:]]{0,}-[[:graph:]]{0,}' 이 옵션으로 전처리를 하면 문제발생함
# ==> 하이픈과 그 전후로 있는 문구를 다같이 삭제하므로 원본데이터에 영향을 미침
# pattern = '-' ==> 하이픈이 사용된 부문만을 탐색해 전처리할 수 있도록 함

class(my_pp_pmnsc)
# - stringr패키지의 문자열 전처리 함수인 str_replace_all을 이용했으므로 
#   전처리결과도 코퍼스(말뭉치) 객체형식이 사라지고 일반 리스트 객체형식으로 변형됨

# 하이픈기호가 사용된 문구는 전처리되어 해당되는 사항이 없음
sapply(my_pp_pmnsc , str_extract_all, pattern = '[[:graph:]]{0,}-[[:graph:]]{0,}')  %>% 
    unlist %>% table %>% sort(decreasing = TRUE) %>% print

[1] "character"

integer(0)


In [93]:
# 하이픈기호가 사용된 문구는 전처리되어 해당되는 사항이 없음

sprintf('전체 말뭉치에서 문장부호&특수문자 패턴 출현내용을 빈도수 기준 내림차순으로 정렬:')
my_pp_tb <- sapply(my_pp_pmnsc, str_extract_all, pattern = '[[:graph:]]{0,}[[:punct:]]{1,}[[:graph:]]{0,}') %>% 
    unlist %>% table %>% sort(decreasing = TRUE) 
my_pp_df <- data.frame(term = names(my_pp_tb), freq = c(my_pp_tb), row.names = NULL)
my_pp_df %>% dplyr::arrange(desc(freq), term) %>% print

[1] "전체 말뭉치에서 문장부호&특수문자 패턴 출현내용을 빈도수 기준 내림차순으로 정렬:"

                             term freq
1                              ()   22
2                      discussed.    9
3                          study,    6
4                          (i.e.,    5
5                         users’    5
6                       findings,    4
7                        however,    4
8                  participation.    4
9                          (e.g.,    3
10                             (n    3
11                          (sns)    3
12                     addiction.    3
13                      citizens'    3
14                    conditions.    3
15                      emotions,    3
16                       people's    3
17                       privacy,    3
18                    skepticism,    3
19                         snss’    3
20                            (a)    2
21                            (b)    2
22               (operationalized    2
23                          (sem)    2
24                             ).    2
25                    activ

## 문구에 슬래시( / ) 표시가 있는 경우

### 문구에 슬래시( / ) 표시가 있는 패턴탐색

In [94]:
# 전체 말뭉치에서 문장부호&특수문자 패턴 중 특정한 출현내용을 별도로 확인
my_pp_df %>% arrange(desc(freq), term) %>% 
    filter(str_detect(term, pattern = '[[:graph:]]{0,}/[[:graph:]]{0,}')) %>% print

# 정규표현식
# - 양쪽 [[:graph:]]{0,} ==> 슬래시 기호 양쪽에 연결되어 있는 문구확인 ==> [[:graph:]]* 동일검색패턴

# 전처리 방향
# - 슬래시( / ) 기호가 사용된 문구는 양쪽 단어의 독립성이 인정되므로 
#   슬래시( / ) 기호를 공백으로 처리하여 두 개 단어로 분리하도록 함

           term freq
1        and/or    2
2 export/import    1
3       his/her    1
4  news/opinion    1
5 news/opinion.    1
6 news/opinions    1


### tm패키지에 일반함수를 이용한 전처리

In [95]:
# tm::tm_map()에 속해 있는 전용 전처리 함수가 아니라 일반 전처리 함수를 이용한 방식 
# - 이 결과로 코퍼스(말뭉치)객체형식이 유지되어 
#   tm패키지 전처리함수 적용을 추가적으로 계속할 수 있음

my_pp_pmsc <- tm_map(my_pp_pmsc, content_transformer(str_replace_all), 
                     pattern = '/', replacement = ' ')
# - pmsc: punctuation marks & special character
# - 앞서 문장부호&특수기호 중 하이픈기호가 전처리된 my_pp_pmsc 코퍼스객체를 사용함
# - 코퍼스 각 문서별로 포함되어 있는 슬래시기호는 삭제하고, 전후로 연결된 단어를 독립적 단어들로 분리함

# (주의!!!)
# pattern = '[[:graph:]]{0,}/[[:graph:]]{0,}' 이 옵션으로 전처리를 하면 문제발생함
# ==> 슬래시와 그 전후로 있는 문구를 다같이 삭제하므로 원본데이터에 영향을 미침
# pattern = '/' ==> 슬래시 기호가 사용된 부문만을 탐색해 전처리할 수 있도록 함

class(my_pp_pmsc)
# - tm패키지 tm_map()에 일반 함수를 사용가능하도록 해주는 content_transformer를 이용했으므로
#   전처리결과도 코퍼스(말뭉치) 객체형식을 유지하고 있음

# 슬래시기호가 사용된 문구는 전처리되어 해당되는 사항이 없음
sapply(my_pp_pmsc , str_extract_all, pattern = '[[:graph:]]{0,}/[[:graph:]]{0,}')  %>% 
    unlist %>% table %>% sort(decreasing = TRUE) %>% print

[1] "VCorpus" "Corpus"

integer(0)


In [96]:
# 슬래시기호가 사용된 문구는 전처리되어 해당되는 사항이 없음

sprintf('전체 말뭉치에서 문장부호&특수문자 패턴 출현내용을 빈도수 기준 내림차순으로 정렬:')
my_pp_tb <- sapply(my_pp_pmsc, str_extract_all, pattern = '[[:graph:]]{0,}[[:punct:]]{1,}[[:graph:]]{0,}') %>% 
    unlist %>% table %>% sort(decreasing = TRUE) 
my_pp_df <- data.frame(term = names(my_pp_tb), freq = c(my_pp_tb), row.names = NULL)
my_pp_df %>% dplyr::arrange(desc(freq), term) %>% print

[1] "전체 말뭉치에서 문장부호&특수문자 패턴 출현내용을 빈도수 기준 내림차순으로 정렬:"

                             term freq
1                              ()   22
2                      discussed.    9
3                          study,    6
4                          (i.e.,    5
5                         users’    5
6                       findings,    4
7                        however,    4
8                  participation.    4
9                          (e.g.,    3
10                             (n    3
11                          (sns)    3
12                     addiction.    3
13                      citizens'    3
14                    conditions.    3
15                      emotions,    3
16                       people's    3
17                       privacy,    3
18                    skepticism,    3
19                         snss’    3
20                            (a)    2
21                            (b)    2
22               (operationalized    2
23                          (sem)    2
24                             ).    2
25                    activ

### stringr패키지를 이용한 전처리

In [97]:
# stringr::str_replace_all() 함수를 이용한 전처리
# - 이 결과로 코퍼스(말뭉치)객체가 문자열객체로 변환되어, 
#   tm패키지 전처리함수 적용을 하려면 다시 말뭉치객체로 만드는 작업이 필요함

my_pp_pmnsc <- sapply(my_pp_pmnsc, str_replace_all, pattern = '/', replacement = ' ')
# - pmsc: punctuation marks and special character
# - 앞서 문장부호&특수기호 중 하이픈기호가 전처리된 my_pp_pmnsc 코퍼스객체를 사용함
# - 코퍼스 각 문서별로 포함되어 있는 슬래시기호는 삭제하고, 전후로 연결된 단어를 독립적 단어들로 분리함

# (주의!!!)
# pattern = '[[:graph:]]{0,}/[[:graph:]]{0,}' 이 옵션으로 전처리를 하면 문제발생함
# ==> 슬래시기호와 그 전후로 있는 문구를 다같이 삭제하므로 원본데이터에 영향을 미침
# pattern = '/' ==> 슬래시가 사용된 부문만을 탐색해 전처리할 수 있도록 함

class(my_pp_pmnsc)
# - stringr패키지의 문자열 전처리 함수인 str_replace_all을 이용했으므로 
#   전처리결과도 코퍼스(말뭉치) 객체형식이 사라지고 일반 리스트 객체형식으로 변형됨

# 슬래시기호가 사용된 문구는 전처리되어 해당되는 사항이 없음
sapply(my_pp_pmnsc , str_extract_all, pattern = '[[:graph:]]{0,}/[[:graph:]]{0,}')  %>% 
    unlist %>% table %>% sort(decreasing = TRUE) %>% print

[1] "character"

integer(0)


In [98]:
# 슬래시기호가 사용된 문구는 전처리되어 해당되는 사항이 없음

sprintf('전체 말뭉치에서 문장부호&특수문자 패턴 출현내용을 빈도수 기준 내림차순으로 정렬:')
my_pp_tb <- sapply(my_pp_pmnsc, str_extract_all, pattern = '[[:graph:]]{0,}[[:punct:]]{1,}[[:graph:]]{0,}') %>% 
    unlist %>% table %>% sort(decreasing = TRUE) 
my_pp_df <- data.frame(term = names(my_pp_tb), freq = c(my_pp_tb), row.names = NULL)
my_pp_df %>% dplyr::arrange(desc(freq), term) %>% print

[1] "전체 말뭉치에서 문장부호&특수문자 패턴 출현내용을 빈도수 기준 내림차순으로 정렬:"

                             term freq
1                              ()   22
2                      discussed.    9
3                          study,    6
4                          (i.e.,    5
5                         users’    5
6                       findings,    4
7                        however,    4
8                  participation.    4
9                          (e.g.,    3
10                             (n    3
11                          (sns)    3
12                     addiction.    3
13                      citizens'    3
14                    conditions.    3
15                      emotions,    3
16                       people's    3
17                       privacy,    3
18                    skepticism,    3
19                         snss’    3
20                            (a)    2
21                            (b)    2
22               (operationalized    2
23                          (sem)    2
24                             ).    2
25                    activ

## 문구에 어포스트로피(단어's, 단어’s) 표시가 있는 경우

### 문구에 어포스트로피(단어's, 단어’s) 표시가 있는 패턴탐색

In [99]:
# 전체 말뭉치에서 문장부호&특수문자 패턴 중 특정한 출현내용을 별도로 확인
my_pp_df %>% arrange(desc(freq), term) %>% 
    filter(str_detect(term, pattern = '[[:graph:]]{1,}((\'s)|(’s))\\b')) %>% print

# 정규표현식
# - 왼쪽 [[:graph:]]{1,} ==> 어포스트로피(단어's, 단어’s) 이전에 어떠한 문구가 오는지 검색패턴
# - 오른쪽 (\'|’|\'s|’s)\\b') ==> 어포스트로피(단어's, 단어’s) 표현으로 끝나는 문구 검색패턴
# - \\b는 바운더리의 개념 ->  여기까지라고 표현하기 위함 그냥 b만 쓰면 글자 'b'를 찾기 때문에 \\b로 작성해줘야 한다.

# 전처리 방향
# - 특별히 어포스트로피 표현자체가 중요한 고유명사로서의 의미가 있는 경우는 없는 것으로 파악되어  
#   어포스트로피 표현만을 일괄 삭제하는 것으로 처리하기로 함

         term freq
1    people's    3
2  hofstede's    2
3 gudykunst's    1
4       isp's    1
5  message’s    1
6     other's    1
7   people’s    1


### tm패키지에 일반함수를 이용한 전처리

In [100]:
# tm::tm_map()에 속해 있는 전용 전처리 함수가 아니라 일반 전처리 함수를 이용한 방식 
# - 이 결과로 코퍼스(말뭉치)객체형식이 유지되어 
#   tm패키지 전처리함수 적용을 추가적으로 계속할 수 있음

my_pp_pmsc <- tm_map(my_pp_pmsc, content_transformer(str_replace_all), 
                     pattern = '\\b((\'s)|(’s))\\b', replacement = ' ')
# - pmsc: punctuation marks & special character
# - 앞서 문장부호&특수기호 중 슬래시기호가 전처리된 my_pp_pmsc 코퍼스객체를 사용함
# - 코퍼스 각 문서별로 포함되어 있는 어포스트로피표현은 삭제하는 것으로 처리함

# (주의!!!)
# pattern = '[[:graph:]]{1,}((\'s)|(’s))\\b' 이 옵션으로 전처리를 하면 문제발생함
# ==> 어포스트로피 표시와 그 앞에 있는 문구를 다같이 삭제하므로 원본데이터에 영향을 미침
# pattern = '\\b(\'|’|\'s|’s)\\b' ==> 어포스트포리 표현이 사용된 부문만을 
#           단어경계로 설정하여 탐색해 전처리할 수 있도록 함

class(my_pp_pmsc)
# - tm패키지 tm_map()에 일반 함수를 사용가능하도록 해주는 content_transformer를 이용했으므로
#   전처리결과도 코퍼스(말뭉치) 객체형식을 유지하고 있음

# 어포스트로피 표현이 사용된 문구는 전처리되어 해당되는 사항이 없음
sapply(my_pp_pmsc , str_extract_all, pattern = '[[:graph:]]{1,}((\'s)|(’s))\\b')  %>% 
    unlist %>% table %>% sort(decreasing = TRUE) %>% print

[1] "VCorpus" "Corpus"

integer(0)


In [101]:
# 어포스트로피 표현이 사용된 문구는 전처리되어 해당되는 사항이 없음

sprintf('전체 말뭉치에서 문장부호&특수문자 패턴 출현내용을 빈도수 기준 내림차순으로 정렬:')
my_pp_tb <- sapply(my_pp_pmsc, str_extract_all, pattern = '[[:graph:]]{0,}[[:punct:]]{1,}[[:graph:]]{0,}') %>% 
    unlist %>% table %>% sort(decreasing = TRUE) 
my_pp_df <- data.frame(term = names(my_pp_tb), freq = c(my_pp_tb), row.names = NULL)
my_pp_df %>% dplyr::arrange(desc(freq), term) %>% print

[1] "전체 말뭉치에서 문장부호&특수문자 패턴 출현내용을 빈도수 기준 내림차순으로 정렬:"

                             term freq
1                              ()   22
2                      discussed.    9
3                          study,    6
4                          (i.e.,    5
5                         users’    5
6                       findings,    4
7                        however,    4
8                  participation.    4
9                          (e.g.,    3
10                             (n    3
11                          (sns)    3
12                     addiction.    3
13                      citizens'    3
14                    conditions.    3
15                      emotions,    3
16                       privacy,    3
17                    skepticism,    3
18                         snss’    3
19                            (a)    2
20                            (b)    2
21               (operationalized    2
22                          (sem)    2
23                             ).    2
24                    activities,    2
25                        a

### stringr패키지를 이용한 전처리

In [102]:
# stringr::str_replace_all() 함수를 이용한 전처리
# - 이 결과로 코퍼스(말뭉치)객체가 문자열객체로 변환되어, 
#   tm패키지 전처리함수 적용을 하려면 다시 말뭉치객체로 만드는 작업이 필요함

my_pp_pmnsc <- sapply(my_pp_pmnsc, str_replace_all, pattern = '\\b(\'s|’s)\\b', replacement = '')
# - pmsc: punctuation marks and special character
# - 앞서 문장부호&특수기호 중 슬래시기호가 전처리된 my_pp_pmsc 코퍼스객체를 사용함
# - 코퍼스 각 문서별로 포함되어 있는 어포스트로피표현은 삭제하는 것으로 처리함

# (주의!!!)
# pattern = '[[:graph:]]{1,}(\'s|’s)\\b' 이 옵션으로 전처리를 하면 문제발생함
# ==> 어포스트로피(단어's 또는 단어’s) 표시와 그 앞에 있는 문구를 다같이 삭제하므로 원본데이터에 영향을 미침
# pattern = '\\b(\'s|’s)\\b' ==> 어포스트로피(단어's 또는 단어’s) 표현이 사용된 부문만을 단어경계로 설정하여 탐색해 전처리할 수 있도록 함

class(my_pp_pmnsc)
# - stringr패키지의 문자열 전처리 함수인 str_replace_all을 이용했으므로 
#   전처리결과도 코퍼스(말뭉치) 객체형식이 사라지고 일반 리스트 객체형식으로 변형됨

# 어포스트로피 표현이 사용된 문구는 전처리되어 해당되는 사항이 없음
sapply(my_pp_pmnsc , str_extract_all, pattern = '[[:graph:]]{1,}(\'s|’s)\\b') %>% 
    unlist %>% table %>% sort(decreasing = TRUE) %>% print

[1] "character"

integer(0)


In [103]:
# 어포스트로피 표현이 사용된 문구는 전처리되어 해당되는 사항이 없음

sprintf('전체 말뭉치에서 문장부호&특수문자 패턴 출현내용을 빈도수 기준 내림차순으로 정렬:')
my_pp_tb <- sapply(my_pp_pmnsc, str_extract_all, pattern = '[[:graph:]]{0,}[[:punct:]]{1,}[[:graph:]]{0,}') %>% 
    unlist %>% table %>% sort(decreasing = TRUE) 
my_pp_df <- data.frame(term = names(my_pp_tb), freq = c(my_pp_tb), row.names = NULL)
my_pp_df %>% dplyr::arrange(desc(freq), term) %>% print

[1] "전체 말뭉치에서 문장부호&특수문자 패턴 출현내용을 빈도수 기준 내림차순으로 정렬:"

                             term freq
1                              ()   22
2                      discussed.    9
3                          study,    6
4                          (i.e.,    5
5                         users’    5
6                       findings,    4
7                        however,    4
8                  participation.    4
9                          (e.g.,    3
10                             (n    3
11                          (sns)    3
12                     addiction.    3
13                      citizens'    3
14                    conditions.    3
15                      emotions,    3
16                       privacy,    3
17                    skepticism,    3
18                         snss’    3
19                            (a)    2
20                            (b)    2
21               (operationalized    2
22                          (sem)    2
23                             ).    2
24                    activities,    2
25                        a

## 문구에 점물음표(.?) 표시가 있는 경우

### 문구에 점물음표(.?) 표시가 있는 패턴탐색

In [104]:
# 전체 말뭉치에서 문장부호&특수문자 패턴 중 특정한 출현내용을 별도로 확인
my_pp_df %>% arrange(desc(freq), term) %>% 
    filter(str_detect(term, pattern = '[[:graph:]]+(\\.\\?)[[:graph:]]+')) %>% print

# 정규표현식
# - 가운데 (\\.\\?) ==> 점(.)과 물음표(?)로 이어지는 문구가 오는지 검색패턴
# - 양쪽  [[:graph:]]+ ==> 점(.)과 물음표(?)로 이어지는 문구 전후로 어떠한 문구가 오는지 검색패턴

# 전처리 방향
# - 점물음표 표현자체가 중요한 고유명사로서의 의미가 있는 경우는 없는 것으로 파악되어  
#   점물음표 문구 표현만을 일괄 삭제하여 전후로 연결된 단어을 독립적으로 분리하는 것으로 전처리함

                 term freq
1     commitment.?the    2
2   commitment.?using    1
3 plausible.?findings    1


### tm패키지에 일반함수를 이용한 전처리

In [105]:
# tm::tm_map()에 속해 있는 전용 전처리 함수가 아니라 일반 전처리 함수를 이용한 방식 
# - 이 결과로 코퍼스(말뭉치)객체형식이 유지되어 
#   tm패키지 전처리함수 적용을 추가적으로 계속할 수 있음

my_pp_pmsc <- tm_map(my_pp_pmsc, content_transformer(str_replace_all), 
                     pattern = '\\b(\\.\\?)\\b', replacement = ' ')
# - pmsc: punctuation marks & special character
# - 앞서 문장부호&특수기호 중 어포스트로피(단어's 또는 단어’s) 표현이 전처리된 my_pp_pmsc 코퍼스객체를 사용함
# - 코퍼스 각 문서별로 포함되어 있는 점물음표 표현은 삭제하는 것으로 처리함

# (주의!!!)
# pattern = '[[:graph:]]+(\\.\\?)[[:graph:]]+' 이 옵션으로 전처리를 하면 문제발생함
# ==> 점물음표 표현 전후로 있는 문구를 다같이 삭제하므로 원본데이터에 영향을 미침
# pattern = '\\b(\\.\\?)\\b' ==> 점물음표 표현이 사용된 부문만을 단어경계로 설정하여 탐색해 전처리할 수 있도록 함

class(my_pp_pmsc)
# - tm패키지 tm_map()에 일반 함수를 사용가능하도록 해주는 content_transformer를 이용했으므로
#   전처리결과도 코퍼스(말뭉치) 객체형식을 유지하고 있음

# 점물음표 표현이 사용된 문구는 전처리되어 해당되는 사항이 없음
sapply(my_pp_pmsc , str_extract_all, pattern = '[[:graph:]]+(\\.\\?)[[:graph:]]+')  %>% 
    unlist %>% table %>% sort(decreasing = TRUE) %>% print

[1] "VCorpus" "Corpus"

integer(0)


In [106]:
# 점물음표 표현이 사용된 문구는 전처리되어 해당되는 사항이 없음

sprintf('전체 말뭉치에서 문장부호&특수문자 패턴 출현내용을 빈도수 기준 내림차순으로 정렬:')
my_pp_tb <- sapply(my_pp_pmsc, str_extract_all, pattern = '[[:graph:]]{0,}[[:punct:]]{1,}[[:graph:]]{0,}') %>% 
    unlist %>% table %>% sort(decreasing = TRUE) 
my_pp_df <- data.frame(term = names(my_pp_tb), freq = c(my_pp_tb), row.names = NULL)
my_pp_df %>% dplyr::arrange(desc(freq), term) %>% print

[1] "전체 말뭉치에서 문장부호&특수문자 패턴 출현내용을 빈도수 기준 내림차순으로 정렬:"

                             term freq
1                              ()   22
2                      discussed.    9
3                          study,    6
4                          (i.e.,    5
5                         users’    5
6                       findings,    4
7                        however,    4
8                  participation.    4
9                          (e.g.,    3
10                             (n    3
11                          (sns)    3
12                     addiction.    3
13                      citizens'    3
14                    conditions.    3
15                      emotions,    3
16                       privacy,    3
17                    skepticism,    3
18                         snss’    3
19                            (a)    2
20                            (b)    2
21               (operationalized    2
22                          (sem)    2
23                             ).    2
24                    activities,    2
25                        a

### stringr패키지를 이용한 전처리

In [107]:
# stringr::str_replace_all() 함수를 이용한 전처리
# - 이 결과로 코퍼스(말뭉치)객체가 문자열객체로 변환되어, 
#   tm패키지 전처리함수 적용을 하려면 다시 말뭉치객체로 만드는 작업이 필요함

my_pp_pmnsc <- sapply(my_pp_pmnsc, str_replace_all, pattern = '\\b(\\.\\?)\\b', replacement = ' ')
# - pmsc: punctuation marks and special character
# - 앞서 문장부호&특수기호 중 어포스트로피(단어's 또는 단어’s) 표현이 전처리된 my_pp_pmnsc 코퍼스객체를 사용함
# - 코퍼스 각 문서별로 포함되어 있는 점물음표 표현은 삭제하는 것으로 처리함

# (주의!!!)
# pattern = '[[:graph:]]+(\\.\\?)[[:graph:]]+' 이 옵션으로 전처리를 하면 문제발생함
# ==> 점물음표 표현 전후로 있는 문구를 다같이 삭제하므로 원본데이터에 영향을 미침
# pattern = '\\b(\\.\\?)\\b' ==> 점물음표 표현이 사용된 부문만을 단어경계로 설정하여 탐색해 전처리할 수 있도록 함

class(my_pp_pmnsc)
# - stringr패키지의 문자열 전처리 함수인 str_replace_all을 이용했으므로 
#   전처리결과도 코퍼스(말뭉치) 객체형식이 사라지고 일반 리스트 객체형식으로 변형됨

# 점물음표 표현이 사용된 문구는 전처리되어 해당되는 사항이 없음
sapply(my_pp_pmnsc , str_extract_all, pattern = '[[:graph:]]+(\\.\\?)[[:graph:]]+')  %>% 
    unlist %>% table %>% sort(decreasing = TRUE) %>% print

[1] "character"

integer(0)


In [108]:
# 점물음표 표현이 사용된 문구는 전처리되어 해당되는 사항이 없음

sprintf('전체 말뭉치에서 문장부호&특수문자 패턴 출현내용을 빈도수 기준 내림차순으로 정렬:')
my_pp_tb <- sapply(my_pp_pmnsc, str_extract_all, pattern = '[[:graph:]]{0,}[[:punct:]]{1,}[[:graph:]]{0,}') %>% 
    unlist %>% table %>% sort(decreasing = TRUE) 
my_pp_df <- data.frame(term = names(my_pp_tb), freq = c(my_pp_tb), row.names = NULL)
my_pp_df %>% dplyr::arrange(desc(freq), term) %>% print

[1] "전체 말뭉치에서 문장부호&특수문자 패턴 출현내용을 빈도수 기준 내림차순으로 정렬:"

                             term freq
1                              ()   22
2                      discussed.    9
3                          study,    6
4                          (i.e.,    5
5                         users’    5
6                       findings,    4
7                        however,    4
8                  participation.    4
9                          (e.g.,    3
10                             (n    3
11                          (sns)    3
12                     addiction.    3
13                      citizens'    3
14                    conditions.    3
15                      emotions,    3
16                       privacy,    3
17                    skepticism,    3
18                         snss’    3
19                            (a)    2
20                            (b)    2
21               (operationalized    2
22                          (sem)    2
23                             ).    2
24                    activities,    2
25                        a

## 문구에 (알파벳1개) 표시가 있는 경우

### 문구에 (a) (b) (c)와 같은 표시가 있는 패턴탐색

In [109]:
# 전체 말뭉치에서 문장부호&특수문자 패턴 중 특정한 출현내용을 별도로 확인
my_pp_df %>% arrange(desc(freq), term) %>% 
    filter(str_detect(term, pattern = '[[:graph:]]{0,}(\\([[:alpha:]]\\))[[:graph:]]{0,}')) %>% print

# 정규표현식
# - 가운데 \\([[:alpha:]]\\) ==> (a), (b), (c) 등의 괄호사이에 영어철자가 1개 오는 문구가 있는지 검색패턴
# - 양쪽  [[:graph:]]{0,} ==> [괄호사이에 영어철자가 1개 오는 문구] 전후로 어떠한 문구가 오는지 검색패턴
#   - [[:graph:]]{0,}는 [[:graph:]]*와 같은 표현. 글자가 아무것고 없어도(0개) 되고 그 이상 여러개 와도 가능

# 전처리 방향
# - 괄호사이에 영어철자가 1개 오는 문구는 중요한 고유명사로서의 의미가 있는 경우는 없는 것으로 파악되어 일괄삭제처리함

  term freq
1  (a)    2
2  (b)    2
3  (c)    1


### tm패키지에 일반함수를 이용한 전처리

In [110]:
# tm::tm_map()에 속해 있는 전용 전처리 함수가 아니라 일반 전처리 함수를 이용한 방식 
# - 이 결과로 코퍼스(말뭉치)객체형식이 유지되어 
#   tm패키지 전처리함수 적용을 추가적으로 계속할 수 있음

my_pp_pmsc <- tm_map(my_pp_pmsc, content_transformer(str_replace_all), 
                     pattern = '\\([[:alpha:]]\\)', replacement = '')
# - pmsc: punctuation marks & special character
# - 앞서 문장부호&특수기호 중 점물음표 표현이 전처리된 my_pp_pmsc 코퍼스객체를 사용함
# - 코퍼스 각 문서별로 포함되어 있는 괄호사이에 영어철자가 1개 오는 문구 표현은 삭제하는 것으로 처리함

# (주의!!!)
# pattern = '[[:graph:]]{0,}(\\([[:alpha:]]\\))[[:graph:]]{0,}' 이 옵션으로 전처리를 하면 문제발생함
# ==> 괄호사이에 영어철자가 1개 오는 문구 표현 전후로 있는 문구를 다같이 삭제하므로 원본데이터에 영향을 미침
# pattern = '\\b(\\([[:alpha:]]\\))\\b' 
#            ==> 괄호사이에 영어철자가 1개 오는 문구표현이 사용된 부문만을 단어경계로 설정하여 탐색해 전처리할 수 있도록 함

class(my_pp_pmsc)
# - tm패키지 tm_map()에 일반 함수를 사용가능하도록 해주는 content_transformer를 이용했으므로
#   전처리결과도 코퍼스(말뭉치) 객체형식을 유지하고 있음

# 괄호사이에 영어철자가 1개 오는 문구 표현이 사용된 문구는 전처리되어 해당되는 사항이 없음
sapply(my_pp_pmsc , str_extract_all, pattern = '[[:graph:]]{0,}(\\([[:alpha:]]\\))[[:graph:]]{0,}')  %>% 
    unlist %>% table %>% sort(decreasing = TRUE) %>% print

[1] "VCorpus" "Corpus"

integer(0)


In [111]:
# 괄호사이에 영어철자가 1개 오는 표현이 사용된 문구는 전처리되어 해당되는 사항이 없음

sprintf('전체 말뭉치에서 문장부호&특수문자 패턴 출현내용을 빈도수 기준 내림차순으로 정렬:')
my_pp_tb <- sapply(my_pp_pmsc, str_extract_all, pattern = '[[:graph:]]{0,}[[:punct:]]{1,}[[:graph:]]{0,}') %>% 
    unlist %>% table %>% sort(decreasing = TRUE) 
my_pp_df <- data.frame(term = names(my_pp_tb), freq = c(my_pp_tb), row.names = NULL)
my_pp_df %>% dplyr::arrange(desc(freq), term) %>% print

[1] "전체 말뭉치에서 문장부호&특수문자 패턴 출현내용을 빈도수 기준 내림차순으로 정렬:"

                             term freq
1                              ()   22
2                      discussed.    9
3                          study,    6
4                          (i.e.,    5
5                         users’    5
6                       findings,    4
7                        however,    4
8                  participation.    4
9                          (e.g.,    3
10                             (n    3
11                          (sns)    3
12                     addiction.    3
13                      citizens'    3
14                    conditions.    3
15                      emotions,    3
16                       privacy,    3
17                    skepticism,    3
18                         snss’    3
19               (operationalized    2
20                          (sem)    2
21                             ).    2
22                    activities,    2
23                        adults.    2
24                   ambivalence,    2
25                     atti

### stringr패키지를 이용한 전처리

In [112]:
# stringr::str_replace_all() 함수를 이용한 전처리
# - 이 결과로 코퍼스(말뭉치)객체가 문자열객체로 변환되어, 
#   tm패키지 전처리함수 적용을 하려면 다시 말뭉치객체로 만드는 작업이 필요함

my_pp_pmnsc <- sapply(my_pp_pmnsc, str_replace_all, pattern = '\\([[:alpha:]]\\)', replacement = '')
# - pmsc: punctuation marks and special character
# - 앞서 문장부호&특수기호 중 점물음표 표현이 전처리된 my_pp_pmsc 코퍼스객체를 사용함
# - 코퍼스 각 문서별로 포함되어 있는 괄호사이에 영어철자가 1개 오는 문구 표현은 삭제하는 것으로 처리함

# (주의!!!)
# pattern = '[[:graph:]]{0,}(\\([[:alpha:]]\\))[[:graph:]]{0,}' 이 옵션으로 전처리를 하면 문제발생함
# ==> 괄호사이에 영어철자가 1개 오는 문구 표현 전후로 있는 문구를 다같이 삭제하므로 원본데이터에 영향을 미침
# pattern = '\\b(\\([[:alpha:]]\\))\\b' 
#            ==> 괄호사이에 영어철자가 1개 오는 문구표현이 사용된 부문만을 단어경계로 설정하여 탐색해 전처리할 수 있도록 함

class(my_pp_pmnsc)
# - stringr패키지의 문자열 전처리 함수인 str_replace_all을 이용했으므로 
#   전처리결과도 코퍼스(말뭉치) 객체형식이 사라지고 일반 리스트 객체형식으로 변형됨

# 괄호사이에 영어철자가 1개 오는 문구 표현이 사용된 문구는 전처리되어 해당되는 사항이 없음
sapply(my_pp_pmnsc , str_extract_all, pattern = '[[:graph:]]{0,}(\\([[:alpha:]]\\))[[:graph:]]{0,}') %>% 
    unlist %>% table %>% sort(decreasing = TRUE) %>% print

[1] "character"

integer(0)


In [113]:
# 괄호사이에 영어철자가 1개 오는 표현이 사용된 문구는 전처리되어 해당되는 사항이 없음

sprintf('전체 말뭉치에서 문장부호&특수문자 패턴 출현내용을 빈도수 기준 내림차순으로 정렬:')
my_pp_tb <- sapply(my_pp_pmnsc, str_extract_all, pattern = '[[:graph:]]{0,}[[:punct:]]{1,}[[:graph:]]{0,}') %>% 
    unlist %>% table %>% sort(decreasing = TRUE) 
my_pp_df <- data.frame(term = names(my_pp_tb), freq = c(my_pp_tb), row.names = NULL)
my_pp_df %>% dplyr::arrange(desc(freq), term) %>% print

[1] "전체 말뭉치에서 문장부호&특수문자 패턴 출현내용을 빈도수 기준 내림차순으로 정렬:"

                             term freq
1                              ()   22
2                      discussed.    9
3                          study,    6
4                          (i.e.,    5
5                         users’    5
6                       findings,    4
7                        however,    4
8                  participation.    4
9                          (e.g.,    3
10                             (n    3
11                          (sns)    3
12                     addiction.    3
13                      citizens'    3
14                    conditions.    3
15                      emotions,    3
16                       privacy,    3
17                    skepticism,    3
18                         snss’    3
19               (operationalized    2
20                          (sem)    2
21                             ).    2
22                    activities,    2
23                        adults.    2
24                   ambivalence,    2
25                     atti

## 특정문구를 직접 다른 문구로 변경

### 정규표현식 방식이 아닌 1:1 직접변환방식으로 문구변경 대상파악

In [114]:
# 전체 말뭉치에서 문장부호&특수문자 패턴 중 1:1 직접변환방식으로 직접변경할 대상

# (i.e., ==> thatis
# (e.g., ==> example
# (n     ==> 삭제
# 둥근따옴표 유형 ‘ ’,“ ” ==> 곧은따옴표 '로 변환

# - 특정한 문구에 대한 전처리시에 정규표현식 사용이 복잡하고, 해당 패턴도 적은 경우에는 
#   1:1 직접변환방식으로 상황에 따라 변경/삭제함 
# - 나머지는 tm패키지의 전용전처리 함수중에서 removePuntuation을 통해서
#   의미 없는 문장부호&특수기호는 일괄삭제하고, 단어는 단어대로 남겨지도록 처리함

### tm패키지에 일반함수를 이용한 전처리

In [115]:
# tm::tm_map()에 속해 있는 전용 전처리 함수가 아니라 일반 전처리 함수를 이용한 방식 
# - 이 결과로 코퍼스(말뭉치)객체형식이 유지되어 
#   tm패키지 전처리함수 적용을 추가적으로 계속할 수 있음

# 1:1 직접변환방식
# - pmsc: punctuation marks & special character
# - 앞서 문장부호&특수기호 중 점물음표 표현이 전처리된 my_pp_pmsc 코퍼스객체를 사용함

my_pp_pmsc <- tm_map(my_pp_pmsc, content_transformer(str_replace_all), 
                     pattern = '\\(i\\.e\\.,', replacement = 'thatis') # (i.e., ==> thatis

my_pp_pmsc <- tm_map(my_pp_pmsc, content_transformer(str_replace_all), 
                     pattern = '\\(e\\.g\\.,', replacement = 'example') # (e.g., ==> example

my_pp_pmsc <- tm_map(my_pp_pmsc, content_transformer(str_replace_all), 
                     pattern = '\\(n', replacement = '') # (n ==> 삭제

my_pp_pmsc <- tm_map(my_pp_pmsc, content_transformer(str_replace_all), 
                     pattern = '(‘|’|“|”)', replacement = '\'') # 둥근따옴표 ’,“, ” ==> 곧은따옴표 '

class(my_pp_pmsc)
# - tm패키지 tm_map()에 일반 함수를 사용가능하도록 해주는 content_transformer를 이용했으므로
#   전처리결과도 코퍼스(말뭉치) 객체형식을 유지하고 있음

# 1:1 직접변환방식 표현이 사용된 문구는 전처리되어 해당되는 사항이 없음
sapply(my_pp_pmsc , str_extract_all, pattern = '(\\(i\\.e\\.,)|(\\(e\\.g\\.,)|(\\(n)|(‘|’|“|”)') %>% 
    unlist %>% table %>% sort(decreasing = TRUE) %>% print

[1] "VCorpus" "Corpus"

integer(0)


In [116]:
# 1:1 직접변환대상 표현이 사용된 문구는 전처리되어 해당되는 사항이 없음

sprintf('전체 말뭉치에서 문장부호&특수문자 패턴 출현내용을 빈도수 기준 내림차순으로 정렬:')
my_pp_tb <- sapply(my_pp_pmsc, str_extract_all, pattern = '[[:graph:]]{0,}[[:punct:]]{1,}[[:graph:]]{0,}') %>% 
    unlist %>% table %>% sort(decreasing = TRUE) 
my_pp_df <- data.frame(term = names(my_pp_tb), freq = c(my_pp_tb), row.names = NULL)
my_pp_df %>% dplyr::arrange(desc(freq), term) %>% print

[1] "전체 말뭉치에서 문장부호&특수문자 패턴 출현내용을 빈도수 기준 내림차순으로 정렬:"

                             term freq
1                              ()   22
2                      discussed.    9
3                          study,    6
4                          users'    6
5                       citizens'    5
6                       findings,    4
7                        however,    4
8                  participation.    4
9                           (sns)    3
10                     addiction.    3
11                    conditions.    3
12                      emotions,    3
13                       privacy,    3
14                    skepticism,    3
15                          snss'    3
16               (operationalized    2
17                          (sem)    2
18                             ).    2
19                    activities,    2
20                        adults.    2
21                   ambivalence,    2
22                     attitudes,    2
23                     audiences'    2
24                     behaviors,    2
25                  conse

### stringr패키지를 이용한 전처리

In [117]:
# stringr::str_replace_all() 함수를 이용한 전처리
# - 이 결과로 코퍼스(말뭉치)객체가 문자열객체로 변환되어, 
#   tm패키지 전처리함수 적용을 하려면 다시 말뭉치객체로 만드는 작업이 필요함

# 1:1 직접변환방식
# - pmnsc: punctuation marks and special character
# - 앞서 문장부호&특수기호 중 점물음표 표현이 전처리된 my_pp_pmnsc 코퍼스객체를 사용함

# (i.e., ==> thatis
my_pp_pmnsc <- sapply(my_pp_pmnsc, str_replace_all, pattern = '\\(i\\.e\\.,', replacement = 'thatis')

# (e.g., ==> example
my_pp_pmnsc <- sapply(my_pp_pmnsc, str_replace_all, pattern = '\\(e\\.g\\.,', replacement = 'example') 

# (n ==> 삭제
my_pp_pmnsc <- sapply(my_pp_pmnsc, str_replace_all, pattern = '\\(n', replacement = '') 

# 둥근따옴표 ’,“, ” ==> 곧은따옴표 '
my_pp_pmnsc <- sapply(my_pp_pmnsc, str_replace_all, pattern = '(‘|’|“|”)', replacement = '\'') 

class(my_pp_pmnsc)
# - stringr패키지의 문자열 전처리 함수인 str_replace_all을 이용했으므로 
#   전처리결과도 코퍼스(말뭉치) 객체형식이 사라지고 일반 리스트 객체형식으로 변형됨

# 1:1 직접변환방식 표현이 사용된 문구는 전처리되어 해당되는 사항이 없음
sapply(my_pp_pmnsc , str_extract_all, pattern = '(\\(i\\.e\\.,)|(\\(e\\.g\\.,)|(\\(n)|(‘|’|“|”)') %>% 
    unlist %>% table %>% sort(decreasing = TRUE) %>% print

[1] "character"

integer(0)


In [118]:
# 1:1 직접변환대상 표현이 사용된 문구는 전처리되어 해당되는 사항이 없음

sprintf('전체 말뭉치에서 문장부호&특수문자 패턴 출현내용을 빈도수 기준 내림차순으로 정렬:')
my_pp_tb <- sapply(my_pp_pmnsc, str_extract_all, pattern = '[[:graph:]]{0,}[[:punct:]]{1,}[[:graph:]]{0,}') %>% 
    unlist %>% table %>% sort(decreasing = TRUE) 
my_pp_df <- data.frame(term = names(my_pp_tb), freq = c(my_pp_tb), row.names = NULL)
my_pp_df %>% dplyr::arrange(desc(freq), term) %>% print

[1] "전체 말뭉치에서 문장부호&특수문자 패턴 출현내용을 빈도수 기준 내림차순으로 정렬:"

                             term freq
1                              ()   22
2                      discussed.    9
3                          study,    6
4                          users'    6
5                       citizens'    5
6                       findings,    4
7                        however,    4
8                  participation.    4
9                           (sns)    3
10                     addiction.    3
11                    conditions.    3
12                      emotions,    3
13                       privacy,    3
14                    skepticism,    3
15                          snss'    3
16               (operationalized    2
17                          (sem)    2
18                             ).    2
19                    activities,    2
20                        adults.    2
21                   ambivalence,    2
22                     attitudes,    2
23                     audiences'    2
24                     behaviors,    2
25                  conse

## 남은 문장부호와 특수문자 일괄제거

### 문구에 문장부호&특수문자 포함 패턴탐색

In [119]:
# 전체 말뭉치에서 문장부호&특수문자 패턴 중 특정한 출현내용을 별도로 확인
my_pp_df %>% arrange(desc(freq), term) %>% 
    filter(str_detect(term, pattern = '[[:graph:]]{0,}[[:punct:]]{1,}[[:graph:]]{0,}')) %>% print

# 정규표현식
# 가운데 [[:punct:]]{1,} ==> 문장부호&특수문자에 해당하는 요소가 1개 이상 들어 있는 패턴검색 ==> [[:punct:]]+ 동일검색패턴
# 양쪽   [[:graph:]]{0,} ==> 문장부호&특수문자가 1개이상 있는 패턴 전후로 어떠한 문자가 오는지 패턴검색 ==> [[:graph:]]* 동일검색패턴

# 전처리 방향
# - 남아 있는 문장부호와 특수문자는 특별한 의미가 없고, 순수한 단어분해에 방해가 되므로 일괄 삭제처리함

                             term freq
1                              ()   22
2                      discussed.    9
3                          study,    6
4                          users'    6
5                       citizens'    5
6                       findings,    4
7                        however,    4
8                  participation.    4
9                           (sns)    3
10                     addiction.    3
11                    conditions.    3
12                      emotions,    3
13                       privacy,    3
14                    skepticism,    3
15                          snss'    3
16               (operationalized    2
17                          (sem)    2
18                             ).    2
19                    activities,    2
20                        adults.    2
21                   ambivalence,    2
22                     attitudes,    2
23                     audiences'    2
24                     behaviors,    2
25                  conse

### tm패키지 전용 전처리함수 이용

In [120]:
# tm::tm_map()에 전용 전처리함수 옵션을 이용한 전처리 
# - 이 결과로 코퍼스(말뭉치)객체형식이 유지되어 
#   tm패키지의 전용 전처리함수 옵션적용을 추가적으로 계속할 수 있음

my_pp_out <- tm_map(my_pp_pmsc, removePunctuation)
# - 앞서 1:1 직접변환대상 표현이 전처리된 my_pp_pmsc 코퍼스객체를 사용함
# - 코퍼스 각 문서별로 포함되어 있는 남아 있는 기타 문장부호&특수기호 표현은 삭제하는 것으로 처리함

class(my_pp_out)
# - tm패키지 tm_map()에 전용 전처리함수 옵션인 removePunctuation을 이용했으므로
#   전처리결과도 코퍼스(말뭉치) 객체형식을 유지하고 있음

# 남은 기타 문장부호와 특수문자 문구 표현이 사용된 문구는 전처리되어 해당되는 사항이 없음
sapply(my_pp_out , str_extract_all, pattern = '[[:graph:]]{0,}[[:punct:]]{1,}[[:graph:]]{0,}') %>% 
    unlist %>% table %>% sort(decreasing = TRUE) %>% print

[1] "VCorpus" "Corpus"

integer(0)


In [121]:
# 남은 문장부호&특수문자 표현이 사용된 문구는 전처리되어 해당되는 사항이 없음

sprintf('전체 말뭉치에서 문장부호&특수문자 패턴 출현내용을 빈도수 기준 내림차순으로 정렬:')
my_pp_tb <- sapply(my_pp_out, str_extract_all, pattern = '[[:graph:]]{0,}[[:punct:]]{1,}[[:graph:]]{0,}') %>% 
    unlist %>% table %>% sort(decreasing = TRUE) 
my_pp_df <- data.frame(term = names(my_pp_tb), freq = c(my_pp_tb), row.names = NULL)
my_pp_df %>% dplyr::arrange(desc(freq), term) %>% print

[1] "전체 말뭉치에서 문장부호&특수문자 패턴 출현내용을 빈도수 기준 내림차순으로 정렬:"

[1] freq
<0 rows> (or 0-length row.names)


### tm패키지에 일반함수를 이용한 전처리

In [122]:
# tm::tm_map()에 속해 있는 전용 전처리 함수가 아니라 일반 전처리 함수를 이용한 방식 
# - 이 결과로 코퍼스(말뭉치)객체형식이 유지되어 
#   tm패키지 전처리함수 적용을 추가적으로 계속할 수 있음

my_pp_result <- tm_map(my_pp_pmsc, content_transformer(str_replace_all), 
                     pattern = '[[:punct:]]{1,}', replacement = '')
# - 앞서 1:1 직접변환대상 표현이 전처리된 my_pp_pmsc 코퍼스객체를 사용함
# - 코퍼스 각 문서별로 포함되어 있는 남아 있는 기타 문장부호&특수기호 표현은 삭제하는 것으로 처리함

# (주의!!!)
# pattern = '[[:graph:]]{0,}(\\([[:alpha:]]\\))[[:graph:]]{0,}' 이 옵션으로 전처리를 하면 문제발생함
# ==> 괄호사이에 영어철자가 1개 오는 문구 표현 전후로 있는 문구를 다같이 삭제하므로 원본데이터에 영향을 미침
# pattern = '[[:punct:]] 
#            ==> 문장부호&특수문자 문구표현이 사용된 부문만을 설정하여 탐색해 전처리할 수 있도록 함

class(my_pp_result)
# - tm패키지 tm_map()에 일반 함수를 사용가능하도록 해주는 content_transformer를 이용했으므로
#   전처리결과도 코퍼스(말뭉치) 객체형식을 유지하고 있음

# 괄호사이에 영어철자가 1개 오는 문구 표현이 사용된 문구는 전처리되어 해당되는 사항이 없음
sapply(my_pp_result , str_extract_all, pattern = '[[:graph:]]{0,}[[:punct:]]{1,}[[:graph:]]{0,}') %>% 
    unlist %>% table %>% sort(decreasing = TRUE) %>% print

[1] "VCorpus" "Corpus"

integer(0)


In [123]:
# 남은 문장부호&특수문자 표현이 사용된 문구는 전처리되어 해당되는 사항이 없음

sprintf('전체 말뭉치에서 문장부호&특수문자 패턴 출현내용을 빈도수 기준 내림차순으로 정렬:')
my_pp_tb <- sapply(my_pp_result, str_extract_all, pattern = '[[:graph:]]{0,}[[:punct:]]{1,}[[:graph:]]{0,}') %>% 
    unlist %>% table %>% sort(decreasing = TRUE) 
my_pp_df <- data.frame(term = names(my_pp_tb), freq = c(my_pp_tb), row.names = NULL)
my_pp_df %>% dplyr::arrange(desc(freq), term) %>% print

[1] "전체 말뭉치에서 문장부호&특수문자 패턴 출현내용을 빈도수 기준 내림차순으로 정렬:"

[1] freq
<0 rows> (or 0-length row.names)


### stringr패키지를 이용한 전처리

In [124]:
# stringr::str_replace_all() 함수를 이용한 전처리
# - 이 결과로 코퍼스(말뭉치)객체가 문자열객체로 변환되어, 
#   tm패키지 전처리함수 적용을 하려면 다시 말뭉치객체로 만드는 작업이 필요함

my_pp_final <- sapply(my_pp_pmnsc, str_replace_all, pattern = '[[:punct:]]{1,}', replacement = '')
# - pmsc: punctuation marks and special character
# - 앞서 문장부호&특수기호 중 점물음표 표현이 전처리된 my_pp_pmsc 코퍼스객체를 사용함
# - 코퍼스 각 문서별로 포함되어 있는 괄호사이에 영어철자가 1개 오는 문구 표현은 삭제하는 것으로 처리함

# (주의!!!)
# pattern = '[[:graph:]]{0,}(\\([[:alpha:]]\\))[[:graph:]]{0,}' 이 옵션으로 전처리를 하면 문제발생함
# ==> 괄호사이에 영어철자가 1개 오는 문구 표현 전후로 있는 문구를 다같이 삭제하므로 원본데이터에 영향을 미침
# pattern = '[[:punct:]] 
#            ==> 문장부호&특수문자 문구표현이 사용된 부문만을 설정하여 탐색해 전처리할 수 있도록 함

class(my_pp_final)
# - stringr패키지의 문자열 전처리 함수인 str_replace_all을 이용했으므로 
#   전처리결과도 코퍼스(말뭉치) 객체형식이 사라지고 일반 리스트 객체형식으로 변형됨

# 괄호사이에 영어철자가 1개 오는 문구 표현이 사용된 문구는 전처리되어 해당되는 사항이 없음
sapply(my_pp_final , str_extract_all, pattern = '[[:graph:]]{0,}[[:punct:]]{1,}[[:graph:]]{0,}') %>% 
    unlist %>% table %>% sort(decreasing = TRUE) %>% print

[1] "character"

integer(0)


In [125]:
# 남은 문장부호&특수문자 표현이 사용된 문구는 전처리되어 해당되는 사항이 없음

sprintf('전체 말뭉치에서 문장부호&특수문자 패턴 출현내용을 빈도수 기준 내림차순으로 정렬:')
my_pp_tb <- sapply(my_pp_final, str_extract_all, pattern = '[[:graph:]]{0,}[[:punct:]]{1,}[[:graph:]]{0,}') %>% 
    unlist %>% table %>% sort(decreasing = TRUE) 
my_pp_df <- data.frame(term = names(my_pp_tb), freq = c(my_pp_tb), row.names = NULL)
my_pp_df %>% dplyr::arrange(desc(freq), term) %>% print

[1] "전체 말뭉치에서 문장부호&특수문자 패턴 출현내용을 빈도수 기준 내림차순으로 정렬:"

[1] freq
<0 rows> (or 0-length row.names)


# <font color = 'blue'>불용단어(stopword) 제거 (p.126)</font>
* 텍스트셋에서 빈번하게 사용되지만 구체적인 의미를 찾기 어려운 단어들을 불용단어라고하며, 보통 삭제를 통해 전처리함
* 영어의 경우 관사(a, an, the), 전치사(in, to, of, for 등), 접속사(and, but, or, as), 한글의 경우 '것, 이, 그, 저' 등의 예가 있음
* 또한 분석가의 판단으로 해당 텍스트셋 문구 중 의미가 없는 단어들을 추가적으로 고려하여 처리함 

<pre></pre>
* <font color = 'red'>tm_map(<font color = 'black'>말뭉치객체</font>, <font color = 'blue'>removeWords</font>, <font color = 'black'>words = 불용어목록</font>)</font>: 말뭉치에 사용된 미리지정한 단어(불용어)를 삭제함

* tm패키지의 불용어 전용 함수를 사용하기 위해서는 대상객체가 코퍼스(말뭉치) 객체유형이어야하며, 
<br> 이를 통해 전처리된 결과객체도 코퍼스(말뭉치) 객체를 유지함

## 불용어 라이브러리 확인
* 텍스트마이닝을 위한 tm패키지에는 지원가능 불용어목록으로 
<br>danish, dutch, english, finnish, french, german, hungarian, italian, norwegian, portuguese, russian, spanish, swedish 등이 가능함
* 영어의 경우 <font color = 'red'>en</font>과 <font color = 'red'>SMART</font>라는 이름으로 불용단어목록이 사전에 정의되어 있음

### tm패키지의 en불용어 목록

In [126]:
sprintf('tm패키지의 en불용어 단어수: %d개', length(stopwords('en')))
sprintf('tm패키지의 en불용어 목록')
stopwords(kind = 'en')
# kind = 'en' 대신에 kind = 'english'라고 입력해도 됨

[1] "tm패키지의 en불용어 단어수: 174개"

[1] "tm패키지의 en불용어 목록"

[1] "i"          "me"         "my"         "myself"     "we"        
  [6] "our"        "ours"       "ourselves"  "you"        "your"      
 [11] "yours"      "yourself"   "yourselves" "he"         "him"       
 [16] "his"        "himself"    "she"        "her"        "hers"      
 [21] "herself"    "it"         "its"        "itself"     "they"      
 [26] "them"       "their"      "theirs"     "themselves" "what"      
 [31] "which"      "who"        "whom"       "this"       "that"      
 [36] "these"      "those"      "am"         "is"         "are"       
 [41] "was"        "were"       "be"         "been"       "being"     
 [46] "have"       "has"        "had"        "having"     "do"        
 [51] "does"       "did"        "doing"      "would"      "should"    
 [56] "could"      "ought"      "i'm"        "you're"     "he's"      
 [61] "she's"      "it's"       "we're"      "they're"    "i've"      
 [66] "you've"     "we've"      "they've"    "i'd"        "you'd"     
 [71] "he'd"       "she'd"      "we'd"       "they'd"     "i'll"      
 [76] "you'll"     "he'll"      "she'll"     "we'll"      "they'll"   
 [81] "isn't"      "aren't"     "wasn't"     "weren't"    "hasn't"    
 [86] "haven't"    "hadn't"     "doesn't"    "don't"      "didn't"    
 [91] "won't"      "wouldn't"   "shan't"     "shouldn't"  "can't"     
 [96] "cannot"     "couldn't"   "mustn't"    "let's"      "that's"    
[101] "who's"      "what's"     "here's"     "there's"    "when's"    
[106] "where's"    "why's"      "how's"      "a"          "an"        
[111] "the"        "and"        "but"        "if"         "or"        
[116] "because"    "as"         "until"      "while"      "of"        
[121] "at"         "by"         "for"        "with"       "about"     
[126] "against"    "between"    "into"       "through"    "during"    
[131] "before"     "after"      "above"      "below"      "to"        
[136] "from"       "up"         "down"       "in"         "out"       
[141] "on"         "off"        "over"       "under"      "again"     
[146] "further"    "then"       "once"       "here"       "there"     
[151] "when"       "where"      "why"        "how"        "all"       
[156] "any"        "both"       "each"       "few"        "more"      
[161] "most"       "other"      "some"       "such"       "no"        
[166] "nor"        "not"        "only"       "own"        "same"      
[171] "so"         "than"       "too"        "very"

### tm패키지의 SMART불용어 목록

In [127]:
sprintf('tm패키지의 SMART불용어 단어수: %d개', length(stopwords('SMART')))
sprintf('tm패키지의 SMART불용어 목록')
stopwords(kind = 'SMART')

[1] "tm패키지의 SMART불용어 단어수: 571개"

[1] "tm패키지의 SMART불용어 목록"

[1] "a"             "a's"           "able"          "about"        
  [5] "above"         "according"     "accordingly"   "across"       
  [9] "actually"      "after"         "afterwards"    "again"        
 [13] "against"       "ain't"         "all"           "allow"        
 [17] "allows"        "almost"        "alone"         "along"        
 [21] "already"       "also"          "although"      "always"       
 [25] "am"            "among"         "amongst"       "an"           
 [29] "and"           "another"       "any"           "anybody"      
 [33] "anyhow"        "anyone"        "anything"      "anyway"       
 [37] "anyways"       "anywhere"      "apart"         "appear"       
 [41] "appreciate"    "appropriate"   "are"           "aren't"       
 [45] "around"        "as"            "aside"         "ask"          
 [49] "asking"        "associated"    "at"            "available"    
 [53] "away"          "awfully"       "b"             "be"           
 [57] "became"        "because"       "become"        "becomes"      
 [61] "becoming"      "been"          "before"        "beforehand"   
 [65] "behind"        "being"         "believe"       "below"        
 [69] "beside"        "besides"       "best"          "better"       
 [73] "between"       "beyond"        "both"          "brief"        
 [77] "but"           "by"            "c"             "c'mon"        
 [81] "c's"           "came"          "can"           "can't"        
 [85] "cannot"        "cant"          "cause"         "causes"       
 [89] "certain"       "certainly"     "changes"       "clearly"      
 [93] "co"            "com"           "come"          "comes"        
 [97] "concerning"    "consequently"  "consider"      "considering"  
[101] "contain"       "containing"    "contains"      "corresponding"
[105] "could"         "couldn't"      "course"        "currently"    
[109] "d"             "definitely"    "described"     "despite"      
[113] "did"           "didn't"        "different"     "do"           
[117] "does"          "doesn't"       "doing"         "don't"        
[121] "done"          "down"          "downwards"     "during"       
[125] "e"             "each"          "edu"           "eg"           
[129] "eight"         "either"        "else"          "elsewhere"    
[133] "enough"        "entirely"      "especially"    "et"           
[137] "etc"           "even"          "ever"          "every"        
[141] "everybody"     "everyone"      "everything"    "everywhere"   
[145] "ex"            "exactly"       "example"       "except"       
[149] "f"             "far"           "few"           "fifth"        
[153] "first"         "five"          "followed"      "following"    
[157] "follows"       "for"           "former"        "formerly"     
[161] "forth"         "four"          "from"          "further"      
[165] "furthermore"   "g"             "get"           "gets"         
[169] "getting"       "given"         "gives"         "go"           
[173] "goes"          "going"         "gone"          "got"          
[177] "gotten"        "greetings"     "h"             "had"          
[181] "hadn't"        "happens"       "hardly"        "has"          
[185] "hasn't"        "have"          "haven't"       "having"       
[189] "he"            "he's"          "hello"         "help"         
[193] "hence"         "her"           "here"          "here's"       
[197] "hereafter"     "hereby"        "herein"        "hereupon"     
[201] "hers"          "herself"       "hi"            "him"          
[205] "himself"       "his"           "hither"        "hopefully"    
[209] "how"           "howbeit"       "however"       "i"            
[213] "i'd"           "i'll"          "i'm"           "i've"         
[217] "ie"            "if"            "ignored"       "immediate"    
[221] "in"            "inasmuch"      "inc"           "indeed"       
[225] "indicate"      "indicated"     "indicates"     "inner"        
[229] "insof

## 불용어 라이브러리 적용

### tm패키지 전용 전처리함수 이용

In [128]:
# tm::tm_map()에 전용 전처리함수 옵션을 이용한 전처리 
# - 이 결과로 코퍼스(말뭉치)객체형식이 유지되어 
#   tm패키지의 전용 전처리함수 옵션적용을 추가적으로 계속할 수 있음

my_pp_rmsw <- tm_map(my_pp_out, removeWords, words = stopwords('SMART'))
# - rmsw: remove stopwords
# - 앞서 남은 기타 문장부호와 특수문자를 일괄제거하는 방식으로 전처리된 my_pp_out 코퍼스객체를 사용함
# - 코퍼스 각 문서별로 포함되어 있는 불용어 표현은 삭제하는 것으로 처리함

class(my_pp_rmsw)
# - tm패키지 tm_map()에 전용 전처리함수 옵션인 removePunctuation을 이용했으므로
#   전처리결과도 코퍼스(말뭉치) 객체형식을 유지하고 있음

[1] "VCorpus" "Corpus"

# <font color = 'blue'>어근동일화(root words identification) 처리 (p.126)</font>
* 텍스트셋에 동일한 단어가 문법적 기능에 따라 다양한 표현으로 사용되는 경우 이를 하나의 통일된 단어로 만들어 주는 것이 필요함
* 영어의 동사는 3인칭인 경우, 시제에 따라서 문법적 표현형태가 다양해짐
* 한국어 동사에서도 가다라는 동사가 가고, 가니, 간 등 맥락에 따라 표현형태가 달라짐

<pre></pre>
* <font color = 'red'>tm_map(<font color = 'black'>말뭉치객체</font>, <font color = 'blue'>stemDocument</font>, <font color = 'black'>language = 'en'</font>)</font>: 말뭉치에 사용된 언어유형에 대한 어근통일을 수행함

* tm패키지의 어근동일화 전용 함수를 사용하기 위해서는 대상객체가 코퍼스(말뭉치) 객체유형이어야하며, 
<br> 이를 통해 전처리된 결과객체도 코퍼스(말뭉치) 객체를 유지함

## 어근동일화 라이브러리 적용

### tm패키지 전용 전처리함수 이용

In [129]:
# tm::tm_map()에 전용 전처리함수 옵션을 이용한 전처리 
# - 이 결과로 코퍼스(말뭉치)객체형식이 유지되어 
#   tm패키지의 전용 전처리함수 옵션적용을 추가적으로 계속할 수 있음

my_pp_rmsd <- tm_map(my_pp_out, stemDocument, language = 'en')
# - rmsd: remove stemdocument
# - 앞서 불용단어를 일괄제거하는 방식으로 전처리된 my_pp_rmsw 코퍼스객체를 사용함
# - 코퍼스 각 문서별로 포함되어 있는 불용어 표현은 삭제하는 것으로 처리함

class(my_pp_rmsd)
# - tm패키지 tm_map()에 전용 전처리함수 옵션인 removePunctuation을 이용했으므로
#   전처리결과도 코퍼스(말뭉치) 객체형식을 유지하고 있음

[1] "VCorpus" "Corpus"

In [ ]:
save(my,my_pp_rmsd, file='mypaper.Rdata')

"""
save(객체1, 객체2, file='저장파일명.Rdata') 

# .Rdata => 바이너리 포맷으로 저장. R에서만 읽기 가능
# .Rdata로 적절히 저장해놓고 다음번에 다시 이 아이를 읽어서 사용하기


cf)
save.image(file='') 
# 지금까지 만든 객체들을 한번에 이미지 바이너리 파일로 저장하겠다.
# 쓸데없는 아무 객체 모두  다 저장됨..ㅎㅎ
"""

# End of Source